In [3]:
import json

response = {
  "type": "message",
  "role": "assistant",
  "content": [
    {
      "type": "text",
      "text": "# Complete MLOps Labs: Understanding Data Drift Concepts with sklearn\n\n## LAB 1: Understanding Data Drift Concepts\n\n```python\n#%% [markdown]\n# # LAB 1: Understanding Data Drift Concepts\n# ## ทำความเข้าใจแนวคิดพื้นฐานของ Data Drift\n# \n# ### วัตถุประสงค์การเรียนรู้:\n# 1. เข้าใจความแตกต่างระหว่าง Covariate Shift และ Concept Drift\n# 2. เรียนรู้ Statistical tests สำหรับ drift detection (KS, PSI, Wasserstein)\n# 3. สามารถเลือก drift detection method ที่เหมาะสมกับสถานการณ์ต่างๆ\n#\n# ### ทฤษฎีพื้นฐาน:\n# **Data Drift** คือ การเปลี่ยนแปลงของข้อมูลเมื่อเวลาผ่านไป ซึ่งอาจส่งผลกระทบต่อประสิทธิภาพของ ML Model\n#\n# มี 2 ประเภทหลัก:\n# - **Covariate Shift**: การเปลี่ยนแปลงของ input features P(X) โดยที่ความสัมพันธ์ P(Y|X) ยังคงเดิม\n# - **Concept Drift**: การเปลี่ยนแปลงของความสัมพันธ์ระหว่าง input และ output P(Y|X)\n\n#%% [markdown]\n# ## ส่วนที่ 1: เตรียม Environment และ Import Libraries\n# \n# ก่อนเริ่มต้น เราจะ import libraries ที่จำเป็นทั้งหมด\n\n#%%\n# Import libraries ที่จำเป็น\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom scipy import stats\nfrom sklearn.datasets import make_classification\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.metrics import accuracy_score, classification_report\nimport warnings\nwarnings.filterwarnings('ignore')\n\n# ตั้งค่า random seed เพื่อให้ผลลัพธ์สามารถทำซ้ำได้\nnp.random.seed(42)\n\n# ตั้งค่า matplotlib สำหรับการแสดงผลภาษาไทย\nplt.rcParams['figure.figsize'] = (12, 6)\nplt.rcParams['font.size'] = 10\n\nprint(\"✅ Libraries imported successfully!\")\nprint(f\"NumPy version: {np.__version__}\")\nprint(f\"Pandas version: {pd.__version__}\")\n\n#%% [markdown]\n# ## ส่วนที่ 2: ทำความเข้าใจ Covariate Shift\n# \n# ### ทฤษฎี Covariate Shift:\n# - เกิดขึ้นเมื่อ distribution ของ input features เปลี่ยนแปลง\n# - ตัวอย่าง: โมเดลทำนายราคาบ้านที่ train กับบ้านในเมือง แต่ต้องทำนายบ้านในชนบท\n# - ความสัมพันธ์ระหว่าง features กับ target ยังคงเหมือนเดิม\n#\n# สูตรทางคณิตศาสตร์:\n# - Training: P_train(X) ≠ P_test(X)\n# - แต่: P(Y|X) คงที่\n\n#%%\ndef generate_covariate_shift_data():\n    \"\"\"\n    สร้างข้อมูลที่แสดง Covariate Shift\n    \n    ในตัวอย่างนี้:\n    - Training data: อายุลูกค้า 20-40 ปี\n    - Production data: อายุลูกค้า 40-60 ปี\n    - ความสัมพันธ์ระหว่างอายุกับพฤติกรรมการซื้อยังคงเดิม\n    \"\"\"\n    \n    # Training data: ลูกค้าอายุน้อย (20-40)\n    np.random.seed(42)\n    n_train = 1000\n    age_train = np.random.normal(30, 5, n_train)\n    income_train = age_train * 1500 + np.random.normal(0, 5000, n_train)\n    \n    # กฎการซื้อ: ซื้อถ้า income > 40000 + age * 500\n    threshold_train = 40000 + age_train * 500\n    purchase_train = (income_train > threshold_train).astype(int)\n    \n    train_df = pd.DataFrame({\n        'age': age_train,\n        'income': income_train,\n        'purchase': purchase_train\n    })\n    \n    # Production data: ลูกค้าอายุมากขึ้น (40-60) - Covariate Shift!\n    n_prod = 1000\n    age_prod = np.random.normal(50, 5, n_prod)  # อายุเปลี่ยน!\n    income_prod = age_prod * 1500 + np.random.normal(0, 5000, n_prod)\n    \n    # กฎการซื้อเหมือนเดิม (ไม่มี Concept Drift)\n    threshold_prod = 40000 + age_prod * 500\n    purchase_prod = (income_prod > threshold_prod).astype(int)\n    \n    prod_df = pd.DataFrame({\n        'age': age_prod,\n        'income': income_prod,\n        'purchase': purchase_prod\n    })\n    \n    return train_df, prod_df\n\n# สร้างข้อมูล\ntrain_covariate, prod_covariate = generate_covariate_shift_data()\n\nprint(\"📊 Training Data Summary:\")\nprint(train_covariate.describe())\nprint(\"\\n📊 Production Data Summary:\")\nprint(prod_covariate.describe())\n\n#%%\n# Visualize Covariate Shift\nfig, axes = plt.subplots(1, 3, figsize=(15, 4))\n\n# Plot 1: Age Distribution\naxes[0].hist(train_covariate['age'], bins=30, alpha=0.7, label='Training', color='blue')\naxes[0].hist(prod_covariate['age'], bins=30, alpha=0.7, label='Production', color='red')\naxes[0].set_xlabel('Age')\naxes[0].set_ylabel('Frequency')\naxes[0].set_title('Covariate Shift: Age Distribution\\n(Training vs Production)')\naxes[0].legend()\naxes[0].axvline(train_covariate['age'].mean(), color='blue', linestyle='--', label='Train Mean')\naxes[0].axvline(prod_covariate['age'].mean(), color='red', linestyle='--', label='Prod Mean')\n\n# Plot 2: Income Distribution\naxes[1].hist(train_covariate['income'], bins=30, alpha=0.7, label='Training', color='blue')\naxes[1].hist(prod_covariate['income'], bins=30, alpha=0.7, label='Production', color='red')\naxes[1].set_xlabel('Income')\naxes[1].set_ylabel('Frequency')\naxes[1].set_title('Covariate Shift: Income Distribution\\n(Training vs Production)')\naxes[1].legend()\n\n# Plot 3: Relationship P(Y|X) ยังคงเดิม\naxes[2].scatter(train_covariate['age'], train_covariate['income'], \n                c=train_covariate['purchase'], alpha=0.3, cmap='coolwarm', label='Training')\naxes[2].set_xlabel('Age')\naxes[2].set_ylabel('Income')\naxes[2].set_title('P(Y|X) Relationship\\n(Decision boundary ยังคงเดิม)')\n\nplt.tight_layout()\nplt.savefig('covariate_shift_visualization.png', dpi=150, bbox_inches='tight')\nplt.show()\n\nprint(\"\\n💡 สังเกต: Distribution ของ Age และ Income เปลี่ยนไป แต่ความสัมพันธ์ในการตัดสินใจซื้อยังเหมือนเดิม\")\n\n#%% [markdown]\n# ## ส่วนที่ 3: ทำความเข้าใจ Concept Drift\n# \n# ### ทฤษฎี Concept Drift:\n# - เกิดขึ้นเมื่อความสัมพันธ์ระหว่าง input และ output เปลี่ยนแปลง\n# - ตัวอย่าง: พฤติกรรมการซื้อของลูกค้าเปลี่ยนหลัง COVID-19\n# - แม้ input distribution จะเหมือนเดิม แต่ output เปลี่ยน\n#\n# สูตรทางคณิตศาสตร์:\n# - P(X) อาจคงที่หรือเปลี่ยนก็ได้\n# - แต่: P(Y|X) เปลี่ยนแปลง\n\n#%%\ndef generate_concept_drift_data():\n    \"\"\"\n    สร้างข้อมูลที่แสดง Concept Drift\n    \n    ในตัวอย่างนี้:\n    - Training data: ลูกค้าซื้อเมื่อ income สูงกว่า threshold\n    - Production data: threshold เปลี่ยน (พฤติกรรมเปลี่ยน)\n    \"\"\"\n    \n    np.random.seed(42)\n    n_samples = 1000\n    \n    # Training data\n    age_train = np.random.normal(35, 10, n_samples)\n    income_train = np.random.normal(50000, 15000, n_samples)\n    \n    # กฎการซื้อเดิม: ซื้อถ้า income > 45000\n    purchase_train = (income_train > 45000).astype(int)\n    \n    train_df = pd.DataFrame({\n        'age': age_train,\n        'income': income_train,\n        'purchase': purchase_train\n    })\n    \n    # Production data: distribution เหมือนเดิม\n    age_prod = np.random.normal(35, 10, n_samples)\n    income_prod = np.random.normal(50000, 15000, n_samples)\n    \n    # กฎการซื้อเปลี่ยน: ซื้อถ้า income > 55000 (Concept Drift!)\n    # อาจเกิดจากเศรษฐกิจตกต่ำ ลูกค้าต้องการรายได้สูงขึ้นถึงจะซื้อ\n    purchase_prod = (income_prod > 55000).astype(int)\n    \n    prod_df = pd.DataFrame({\n        'age': age_prod,\n        'income': income_prod,\n        'purchase': purchase_prod\n    })\n    \n    return train_df, prod_df\n\n# สร้างข้อมูล\ntrain_concept, prod_concept = generate_concept_drift_data()\n\nprint(\"📊 Training Data Summary:\")\nprint(f\"Purchase Rate: {train_concept['purchase'].mean():.2%}\")\nprint(train_concept.describe())\n\nprint(\"\\n📊 Production Data Summary:\")\nprint(f\"Purchase Rate: {prod_concept['purchase'].mean():.2%}\")\nprint(prod_concept.describe())\n\n#%%\n# Visualize Concept Drift\nfig, axes = plt.subplots(1, 3, figsize=(15, 4))\n\n# Plot 1: Income Distribution (เหมือนกัน)\naxes[0].hist(train_concept['income'], bins=30, alpha=0.7, label='Training', color='blue')\naxes[0].hist(prod_concept['income'], bins=30, alpha=0.7, label='Production', color='red')\naxes[0].set_xlabel('Income')\naxes[0].set_ylabel('Frequency')\naxes[0].set_title('No Covariate Shift: Income Distribution\\n(Training ≈ Production)')\naxes[0].legend()\n\n# Plot 2: Purchase Rate by Income Bin\nincome_bins = np.linspace(20000, 80000, 10)\n\ntrain_purchase_rate = []\nprod_purchase_rate = []\nbin_centers = []\n\nfor i in range(len(income_bins)-1):\n    mask_train = (train_concept['income'] >= income_bins[i]) & (train_concept['income'] < income_bins[i+1])\n    mask_prod = (prod_concept['income'] >= income_bins[i]) & (prod_concept['income'] < income_bins[i+1])\n    \n    if mask_train.sum() > 0:\n        train_purchase_rate.append(train_concept[mask_train]['purchase'].mean())\n    else:\n        train_purchase_rate.append(0)\n        \n    if mask_prod.sum() > 0:\n        prod_purchase_rate.append(prod_concept[mask_prod]['purchase'].mean())\n    else:\n        prod_purchase_rate.append(0)\n    \n    bin_centers.append((income_bins[i] + income_bins[i+1]) / 2)\n\naxes[1].plot(bin_centers, train_purchase_rate, 'b-o', label='Training P(Y|X)', linewidth=2)\naxes[1].plot(bin_centers, prod_purchase_rate, 'r-o', label='Production P(Y|X)', linewidth=2)\naxes[1].set_xlabel('Income')\naxes[1].set_ylabel('Purchase Probability')\naxes[1].set_title('Concept Drift: P(Y|X) Changed!\\n(Decision boundary moved)')\naxes[1].legend()\naxes[1].axhline(0.5, color='gray', linestyle='--', alpha=0.5)\n\n# Plot 3: Decision Boundary Comparison\naxes[2].scatter(train_concept['income'], train_concept['purchase'] + np.random.normal(0, 0.05, len(train_concept)), \n                alpha=0.3, color='blue', label='Training')\naxes[2].scatter(prod_concept['income'], prod_concept['purchase'] + np.random.normal(0, 0.05, len(prod_concept)), \n                alpha=0.3, color='red', label='Production')\naxes[2].axvline(45000, color='blue', linestyle='--', linewidth=2, label='Train Threshold (45k)')\naxes[2].axvline(55000, color='red', linestyle='--', linewidth=2, label='Prod Threshold (55k)')\naxes[2].set_xlabel('Income')\naxes[2].set_ylabel('Purchase (with jitter)')\naxes[2].set_title('Concept Drift Visualization\\n(Threshold shifted from 45k to 55k)')\naxes[2].legend()\n\nplt.tight_layout()\nplt.savefig('concept_drift_visualization.png', dpi=150, bbox_inches='tight')\nplt.show()\n\nprint(\"\\n💡 สังเกต: Distribution ของ Income เหมือนเดิม แต่ความสัมพันธ์กับ Purchase เปลี่ยนไป!\")\nprint(\"   - Training: ซื้อเมื่อ income > 45,000\")\nprint(\"   - Production: ซื้อเมื่อ income > 55,000\")\n\n#%% [markdown]\n# ## ส่วนที่ 4: Statistical Tests สำหรับ Drift Detection\n# \n# ### 4.1 Kolmogorov-Smirnov (KS) Test\n# \n# **ทฤษฎี:**\n# - เปรียบเทียบ cumulative distribution function (CDF) ของ 2 samples\n# - วัดความแตกต่างสูงสุดระหว่าง 2 CDFs\n# - ข้อดี: ไม่ต้องสมมติ distribution, sensitive ต่อการเปลี่ยนแปลง\n# - ข้อเสีย: ใช้ได้กับ continuous variables เท่านั้น\n#\n# **การตีความ:**\n# - KS Statistic: 0-1 (ยิ่งสูง = ยิ่งต่าง)\n# - p-value < 0.05: reject null hypothesis → มี drift\n\n#%%\ndef kolmogorov_smirnov_test(data1, data2, feature_name=\"feature\"):\n    \"\"\"\n    ทำ KS Test เพื่อตรวจจับ drift\n    \n    Parameters:\n    -----------\n    data1 : array-like - ข้อมูลชุดแรก (reference/training)\n    data2 : array-like - ข้อมูลชุดที่สอง (current/production)\n    feature_name : str - ชื่อ feature\n    \n    Returns:\n    --------\n    dict : ผลลัพธ์ของ KS test\n    \"\"\"\n    statistic, p_value = stats.ks_2samp(data1, data2)\n    \n    # กำหนด threshold สำหรับ drift\n    drift_detected = p_value < 0.05\n    \n    result = {\n        'feature': feature_name,\n        'test': 'Kolmogorov-Smirnov',\n        'statistic': statistic,\n        'p_value': p_value,\n        'drift_detected': drift_detected,\n        'interpretation': 'DRIFT DETECTED!' if drift_detected else 'No significant drift'\n    }\n    \n    return result\n\n# ทดสอบ KS Test กับข้อมูล Covariate Shift\nprint(\"=\" * 60)\nprint(\"🔍 KS Test Results for Covariate Shift Data\")\nprint(\"=\" * 60)\n\nks_age = kolmogorov_smirnov_test(train_covariate['age'], prod_covariate['age'], 'age')\nks_income = kolmogorov_smirnov_test(train_covariate['income'], prod_covariate['income'], 'income')\n\nfor result in [ks_age, ks_income]:\n    print(f\"\\nFeature: {result['feature']}\")\n    print(f\"  KS Statistic: {result['statistic']:.4f}\")\n    print(f\"  P-value: {result['p_value']:.6f}\")\n    print(f\"  Result: {result['interpretation']}\")\n\n#%%\n# Visualize KS Test\nfig, axes = plt.subplots(1, 2, figsize=(14, 5))\n\nfor idx, (feature, ax) in enumerate(zip(['age', 'income'], axes)):\n    # คำนวณ CDF\n    sorted_train = np.sort(train_covariate[feature])\n    sorted_prod = np.sort(prod_covariate[feature])\n    \n    cdf_train = np.arange(1, len(sorted_train) + 1) / len(sorted_train)\n    cdf_prod = np.arange(1, len(sorted_prod) + 1) / len(sorted_prod)\n    \n    ax.plot(sorted_train, cdf_train, 'b-', linewidth=2, label='Training CDF')\n    ax.plot(sorted_prod, cdf_prod, 'r-', linewidth=2, label='Production CDF')\n    \n    # แสดง KS statistic (maximum distance)\n    ks_result = kolmogorov_smirnov_test(train_covariate[feature], prod_covariate[feature], feature)\n    \n    ax.set_xlabel(feature.capitalize())\n    ax.set_ylabel('Cumulative Probability')\n    ax.set_title(f'KS Test: {feature.capitalize()}\\nKS Statistic = {ks_result[\"statistic\"]:.4f}, p-value = {ks_result[\"p_value\"]:.4f}')\n    ax.legend()\n    ax.grid(True, alpha=0.3)\n\nplt.tight_layout()\nplt.savefig('ks_test_visualization.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%% [markdown]\n# ### 4.2 Population Stability Index (PSI)\n# \n# **ทฤษฎี:**\n# - วัดการเปลี่ยนแปลงของ distribution โดยเปรียบเทียบ proportions ในแต่ละ bin\n# - นิยมใช้ใน credit scoring และ financial models\n# - สูตร: PSI = Σ (Actual% - Expected%) × ln(Actual% / Expected%)\n#\n# **การตีความ PSI:**\n# - PSI < 0.1: ไม่มีการเปลี่ยนแปลงที่สำคัญ\n# - 0.1 ≤ PSI < 0.25: มีการเปลี่ยนแปลงปานกลาง ควรตรวจสอบ\n# - PSI ≥ 0.25: มีการเปลี่ยนแปลงมาก ต้องดำเนินการ\n\n#%%\ndef calculate_psi(expected, actual, bins=10, eps=1e-6):\n    \"\"\"\n    คำนวณ Population Stability Index (PSI)\n    \n    Parameters:\n    -----------\n    expected : array-like - ข้อมูล reference (training)\n    actual : array-like - ข้อมูล current (production)\n    bins : int - จำนวน bins สำหรับ discretize\n    eps : float - ค่าเล็กๆ เพื่อป้องกัน division by zero\n    \n    Returns:\n    --------\n    float : ค่า PSI\n    dict : รายละเอียดการคำนวณ\n    \"\"\"\n    # สร้าง bins จาก expected data\n    breakpoints = np.percentile(expected, np.linspace(0, 100, bins + 1))\n    breakpoints = np.unique(breakpoints)  # ลบ duplicates\n    \n    # นับจำนวนในแต่ละ bin\n    expected_counts, _ = np.histogram(expected, bins=breakpoints)\n    actual_counts, _ = np.histogram(actual, bins=breakpoints)\n    \n    # คำนวณ proportions\n    expected_props = expected_counts / len(expected) + eps\n    actual_props = actual_counts / len(actual) + eps\n    \n    # คำนวณ PSI\n    psi_values = (actual_props - expected_props) * np.log(actual_props / expected_props)\n    psi = np.sum(psi_values)\n    \n    # ตีความผลลัพธ์\n    if psi < 0.1:\n        interpretation = \"No significant change (PSI < 0.1)\"\n        severity = \"LOW\"\n    elif psi < 0.25:\n        interpretation = \"Moderate change - monitor closely (0.1 ≤ PSI < 0.25)\"\n        severity = \"MEDIUM\"\n    else:\n        interpretation = \"Significant change - action required (PSI ≥ 0.25)\"\n        severity = \"HIGH\"\n    \n    return {\n        'psi': psi,\n        'interpretation': interpretation,\n        'severity': severity,\n        'bin_psi_values': psi_values,\n        'expected_props': expected_props,\n        'actual_props': actual_props,\n        'breakpoints': breakpoints\n    }\n\n# คำนวณ PSI สำหรับแต่ละ feature\nprint(\"=\" * 60)\nprint(\"📊 PSI Results for Covariate Shift Data\")\nprint(\"=\" * 60)\n\nfor feature in ['age', 'income']:\n    psi_result = calculate_psi(train_covariate[feature], prod_covariate[feature])\n    print(f\"\\nFeature: {feature}\")\n    print(f\"  PSI Value: {psi_result['psi']:.4f}\")\n    print(f\"  Severity: {psi_result['severity']}\")\n    print(f\"  Interpretation: {psi_result['interpretation']}\")\n\n#%%\n# Visualize PSI\nfig, axes = plt.subplots(1, 2, figsize=(14, 5))\n\nfor idx, feature in enumerate(['age', 'income']):\n    psi_result = calculate_psi(train_covariate[feature], prod_covariate[feature])\n    \n    x = np.arange(len(psi_result['expected_props']))\n    width = 0.35\n    \n    axes[idx].bar(x - width/2, psi_result['expected_props'], width, label='Expected (Train)', color='blue', alpha=0.7)\n    axes[idx].bar(x + width/2, psi_result['actual_props'], width, label='Actual (Prod)', color='red', alpha=0.7)\n    \n    axes[idx].set_xlabel('Bin')\n    axes[idx].set_ylabel('Proportion')\n    axes[idx].set_title(f'PSI Analysis: {feature.capitalize()}\\nPSI = {psi_result[\"psi\"]:.4f} ({psi_result[\"severity\"]})')\n    axes[idx].legend()\n    axes[idx].set_xticks(x)\n\nplt.tight_layout()\nplt.savefig('psi_visualization.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%% [markdown]\n# ### 4.3 Wasserstein Distance (Earth Mover's Distance)\n# \n# **ทฤษฎี:**\n# - วัด \"งาน\" ที่ต้องใช้ในการเปลี่ยน distribution หนึ่งไปเป็นอีก distribution\n# - เหมือนการคำนวณต้นทุนในการขนย้ายดิน (Earth Mover)\n# - ข้อดี: คำนึงถึง distance ระหว่าง bins, sensitive ต่อ shift ในตำแหน่ง\n# - ข้อเสีย: ต้อง normalize ข้อมูลเพื่อให้เปรียบเทียบได้\n\n#%%\ndef wasserstein_distance_test(data1, data2, feature_name=\"feature\"):\n    \"\"\"\n    คำนวณ Wasserstein Distance สำหรับ drift detection\n    \n    Parameters:\n    -----------\n    data1 : array-like - ข้อมูลชุดแรก (reference)\n    data2 : array-like - ข้อมูลชุดที่สอง (current)\n    feature_name : str - ชื่อ feature\n    \n    Returns:\n    --------\n    dict : ผลลัพธ์ของ Wasserstein distance\n    \"\"\"\n    # คำนวณ Wasserstein distance\n    distance = stats.wasserstein_distance(data1, data2)\n    \n    # Normalize โดยใช้ standard deviation ของ reference data\n    std_ref = np.std(data1)\n    normalized_distance = distance / std_ref if std_ref > 0 else distance\n    \n    # กำหนด threshold (ปรับได้ตาม domain)\n    if normalized_distance < 0.1:\n        severity = \"LOW\"\n        interpretation = \"No significant drift\"\n    elif normalized_distance < 0.5:\n        severity = \"MEDIUM\"\n        interpretation = \"Moderate drift detected\"\n    else:\n        severity = \"HIGH\"\n        interpretation = \"Significant drift detected\"\n    \n    return {\n        'feature': feature_name,\n        'distance': distance,\n        'normalized_distance': normalized_distance,\n        'severity': severity,\n        'interpretation': interpretation\n    }\n\n# คำนวณ Wasserstein Distance\nprint(\"=\" * 60)\nprint(\"📏 Wasserstein Distance Results for Covariate Shift Data\")\nprint(\"=\" * 60)\n\nfor feature in ['age', 'income']:\n    wd_result = wasserstein_distance_test(train_covariate[feature], prod_covariate[feature], feature)\n    print(f\"\\nFeature: {feature}\")\n    print(f\"  Wasserstein Distance: {wd_result['distance']:.4f}\")\n    print(f\"  Normalized Distance: {wd_result['normalized_distance']:.4f}\")\n    print(f\"  Severity: {wd_result['severity']}\")\n    print(f\"  Interpretation: {wd_result['interpretation']}\")\n\n#%%\n# Visualize Wasserstein Distance\nfig, axes = plt.subplots(1, 2, figsize=(14, 5))\n\nfor idx, feature in enumerate(['age', 'income']):\n    ax = axes[idx]\n    \n    # แสดง histogram และ KDE\n    ax.hist(train_covariate[feature], bins=30, alpha=0.5, density=True, label='Training', color='blue')\n    ax.hist(prod_covariate[feature], bins=30, alpha=0.5, density=True, label='Production', color='red')\n    \n    # คำนวณ Wasserstein distance\n    wd_result = wasserstein_distance_test(train_covariate[feature], prod_covariate[feature], feature)\n    \n    # แสดง mean และ \"การขนย้าย\"\n    mean_train = train_covariate[feature].mean()\n    mean_prod = prod_covariate[feature].mean()\n    \n    ax.axvline(mean_train, color='blue', linestyle='--', linewidth=2, label=f'Train Mean: {mean_train:.1f}')\n    ax.axvline(mean_prod, color='red', linestyle='--', linewidth=2, label=f'Prod Mean: {mean_prod:.1f}')\n    \n    # แสดง arrow สำหรับ \"earth moving\"\n    ax.annotate('', xy=(mean_prod, 0.01), xytext=(mean_train, 0.01),\n                arrowprops=dict(arrowstyle='->', color='green', lw=3))\n    \n    ax.set_xlabel(feature.capitalize())\n    ax.set_ylabel('Density')\n    ax.set_title(f'Wasserstein Distance: {feature.capitalize()}\\nDistance = {wd_result[\"distance\"]:.2f} ({wd_result[\"severity\"]})')\n    ax.legend(loc='upper right')\n\nplt.tight_layout()\nplt.savefig('wasserstein_visualization.png', dpi=150, bbox_inches='tight')\nplt.show()\n\nprint(\"\\n💡 ลูกศรสีเขียวแสดงทิศทางของ 'การขนย้าย' จาก Training ไป Production\")\n\n#%% [markdown]\n# ## ส่วนที่ 5: เปรียบเทียบและเลือก Drift Detection Method\n# \n# ### สรุปข้อดี-ข้อเสียของแต่ละ method:\n#\n# | Method | ข้อดี | ข้อเสีย | ใช้เมื่อ |\n# |--------|-------|---------|---------|\n# | KS Test | ไม่สมมติ distribution, sensitive | ใช้กับ continuous เท่านั้น | ต้องการ statistical significance |\n# | PSI | มี threshold ชัดเจน, industry standard | ต้อง binning | Credit scoring, Risk models |\n# | Wasserstein | คำนึงถึง distance, เข้าใจง่าย | ต้อง normalize | เปรียบเทียบ shift ในตำแหน่ง |\n\n#%%\ndef comprehensive_drift_analysis(reference_data, current_data, feature_name):\n    \"\"\"\n    วิเคราะห์ drift โดยใช้ทุก methods และเปรียบเทียบผลลัพธ์\n    \"\"\"\n    results = {\n        'feature': feature_name,\n        'ks': kolmogorov_smirnov_test(reference_data, current_data, feature_name),\n        'psi': calculate_psi(reference_data, current_data),\n        'wasserstein': wasserstein_distance_test(reference_data, current_data, feature_name)\n    }\n    \n    # สรุปผล\n    drift_votes = 0\n    if results['ks']['drift_detected']:\n        drift_votes += 1\n    if results['psi']['severity'] in ['MEDIUM', 'HIGH']:\n        drift_votes += 1\n    if results['wasserstein']['severity'] in ['MEDIUM', 'HIGH']:\n        drift_votes += 1\n    \n    results['consensus'] = 'DRIFT' if drift_votes >= 2 else 'NO DRIFT'\n    results['drift_votes'] = drift_votes\n    \n    return results\n\n# เปรียบเทียบผลลัพธ์\nprint(\"=\" * 80)\nprint(\"📊 COMPREHENSIVE DRIFT ANALYSIS COMPARISON\")\nprint(\"=\" * 80)\n\ncomparison_results = []\n\nfor feature in ['age', 'income']:\n    result = comprehensive_drift_analysis(\n        train_covariate[feature], \n        prod_covariate[feature], \n        feature\n    )\n    comparison_results.append(result)\n    \n    print(f\"\\n🔍 Feature: {feature.upper()}\")\n    print(\"-\" * 40)\n    print(f\"  KS Test: stat={result['ks']['statistic']:.4f}, p={result['ks']['p_value']:.4f} → {result['ks']['interpretation']}\")\n    print(f\"  PSI: {result['psi']['psi']:.4f} → {result['psi']['severity']}\")\n    print(f\"  Wasserstein: {result['wasserstein']['normalized_distance']:.4f} → {result['wasserstein']['severity']}\")\n    print(f\"  📋 CONSENSUS ({result['drift_votes']}/3 votes): {result['consensus']}\")\n\n#%%\n# สร้าง comparison table\ncomparison_df = pd.DataFrame([\n    {\n        'Feature': r['feature'],\n        'KS Statistic': f\"{r['ks']['statistic']:.4f}\",\n        'KS p-value': f\"{r['ks']['p_value']:.4f}\",\n        'KS Drift': '✓' if r['ks']['drift_detected'] else '✗',\n        'PSI': f\"{r['psi']['psi']:.4f}\",\n        'PSI Severity': r['psi']['severity'],\n        'Wasserstein': f\"{r['wasserstein']['normalized_distance']:.4f}\",\n        'WD Severity': r['wasserstein']['severity'],\n        'Consensus': r['consensus']\n    }\n    for r in comparison_results\n])\n\nprint(\"\\n\" + \"=\" * 80)\nprint(\"📊 SUMMARY TABLE\")\nprint(\"=\" * 80)\nprint(comparison_df.to_string(index=False))\n\n#%% [markdown]\n# ## ส่วนที่ 6: แนวทางการเลือก Drift Detection Method\n# \n# ### Decision Tree สำหรับเลือก Method:\n#\n# ```\n# 1. Data Type?\n#    ├── Continuous → ไปข้อ 2\n#    └── Categorical → ใช้ Chi-squared test หรือ PSI\n#\n# 2. ต้องการ Statistical Significance?\n#    ├── ใช่ → KS Test หรือ Chi-squared\n#    └── ไม่จำเป็น → PSI หรือ Wasserstein\n#\n# 3. Industry Requirement?\n#    ├── Finance/Credit → PSI (มี regulatory standards)\n#    └── อื่นๆ → เลือกตามความเหมาะสม\n#\n# 4. ต้องการความ sensitive สูง?\n#    ├── ใช่ → Wasserstein (detect small shifts)\n#    └── ปกติ → KS หรือ PSI\n# ```\n\n#%%\ndef recommend_drift_method(data_type, needs_significance, industry, high_sensitivity):\n    \"\"\"\n    แนะนำ drift detection method ที่เหมาะสม\n    \n    Parameters:\n    -----------\n    data_type : str - 'continuous' หรือ 'categorical'\n    needs_significance : bool - ต้องการ statistical significance หรือไม่\n    industry : str - 'finance', 'healthcare', 'general'\n    high_sensitivity : bool - ต้องการ sensitivity สูงหรือไม่\n    \n    Returns:\n    --------\n    str : recommended method พร้อมเหตุผล\n    \"\"\"\n    recommendations = []\n    \n    if data_type == 'categorical':\n        recommendations.append({\n            'method': 'Chi-squared Test',\n            'reason': 'เหมาะกับ categorical data',\n            'priority': 1\n        })\n        recommendations.append({\n            'method': 'PSI (binned)',\n            'reason': 'สามารถใช้กับ categorical ได้โดยใช้ category เป็น bins',\n            'priority': 2\n        })\n    else:  # continuous\n        if industry == 'finance':\n            recommendations.append({\n                'method': 'PSI',\n                'reason': 'Industry standard ใน finance, มี regulatory requirements',\n                'priority': 1\n            })\n        \n        if needs_significance:\n            recommendations.append({\n                'method': 'KS Test',\n                'reason': 'ให้ p-value สำหรับ statistical significance',\n                'priority': 1 if industry != 'finance' else 2\n            })\n        \n        if high_sensitivity:\n            recommendations.append({\n                'method': 'Wasserstein Distance',\n                'reason': 'Sensitive ต่อ small shifts และ location changes',\n                'priority': 2\n            })\n        \n        # Default recommendation\n        if not recommendations:\n            recommendations.append({\n                'method': 'PSI + KS Test',\n                'reason': 'ใช้ทั้งสองเพื่อ cross-validate',\n                'priority': 1\n            })\n    \n    # เรียงตาม priority\n    recommendations.sort(key=lambda x: x['priority'])\n    \n    return recommendations\n\n# ตัวอย่างการใช้งาน\nprint(\"=\" * 60)\nprint(\"🎯 DRIFT METHOD RECOMMENDATION EXAMPLES\")\nprint(\"=\" * 60)\n\nscenarios = [\n    {'data_type': 'continuous', 'needs_significance': True, 'industry': 'finance', 'high_sensitivity': False},\n    {'data_type': 'continuous', 'needs_significance': False, 'industry': 'general', 'high_sensitivity': True},\n    {'data_type': 'categorical', 'needs_significance': True, 'industry': 'healthcare', 'high_sensitivity': False},\n]\n\nfor i, scenario in enumerate(scenarios, 1):\n    print(f\"\\n📋 Scenario {i}:\")\n    print(f\"   Data Type: {scenario['data_type']}\")\n    print(f\"   Needs Significance: {scenario['needs_significance']}\")\n    print(f\"   Industry: {scenario['industry']}\")\n    print(f\"   High Sensitivity: {scenario['high_sensitivity']}\")\n    \n    recs = recommend_drift_method(**scenario)\n    print(\"   Recommendations:\")\n    for j, rec in enumerate(recs, 1):\n        print(f\"   {j}. {rec['method']} - {rec['reason']}\")\n\n#%% [markdown]\n# ## สรุป LAB 1\n# \n# ### สิ่งที่เรียนรู้:\n# 1. **Covariate Shift**: P(X) เปลี่ยน แต่ P(Y|X) คงที่\n# 2. **Concept Drift**: P(Y|X) เปลี่ยน (relationship เปลี่ยน)\n# 3. **KS Test**: วัด maximum distance ระหว่าง CDFs, ให้ p-value\n# 4. **PSI**: วัดการเปลี่ยนแปลงของ proportions, มี threshold ชัดเจน\n# 5. **Wasserstein**: วัด \"งาน\" ในการเปลี่ยน distribution\n# \n# ### Best Practices:\n# - ใช้หลาย methods เพื่อ cross-validate\n# - เลือก method ตาม data type และ business requirements\n# - ตั้ง threshold ที่เหมาะสมกับ context\n\n#%%\nprint(\"=\" * 60)\nprint(\"✅ LAB 1 COMPLETED!\")\nprint(\"=\" * 60)\nprint(\"\"\"\n📚 Key Takeaways:\n1. Covariate Shift vs Concept Drift - เข้าใจความแตกต่าง\n2. KS Test - ใช้เมื่อต้องการ statistical significance\n3. PSI - Industry standard สำหรับ finance\n4. Wasserstein - Sensitive ต่อ location shifts\n\n🔜 Next: LAB 2 - Feature Drift Detection\n\"\"\")\n```\n\n---\n\n## LAB 2: Feature Drift Detection\n\n```python\n#%% [markdown]\n# # LAB 2: Feature Drift Detection\n# ## การตรวจจับ Drift ในแต่ละ Feature\n# \n# ### วัตถุประสงค์การเรียนรู้:\n# 1. ตรวจจับ drift ในแต่ละ feature อย่างเป็นระบบ\n# 2. วิเคราะห์ numerical vs categorical feature drift\n# 3. สร้าง visualization สำหรับ feature distributions over time\n#\n# ### ทฤษฎี:\n# การตรวจจับ drift ในแต่ละ feature มีความสำคัญเพราะ:\n# - ช่วยระบุ root cause ของ model performance degradation\n# - ทำให้เข้าใจว่า feature ไหนเปลี่ยนแปลงมากที่สุด\n# - สามารถ prioritize การแก้ไขได้\n\n#%% [markdown]\n# ## ส่วนที่ 1: เตรียม Environment\n\n#%%\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom scipy import stats\nfrom sklearn.datasets import make_classification\nfrom sklearn.preprocessing import StandardScaler\nimport warnings\nwarnings.filterwarnings('ignore')\n\nnp.random.seed(42)\nplt.rcParams['figure.figsize'] = (14, 8)\n\nprint(\"✅ Libraries imported successfully!\")\n\n#%% [markdown]\n# ## ส่วนที่ 2: สร้าง Dataset ที่มีหลาย Features\n# \n# เราจะสร้าง dataset ที่จำลองสถานการณ์จริง:\n# - มีทั้ง numerical และ categorical features\n# - บาง features มี drift บาง features ไม่มี\n# - มี drift ในระดับต่างๆ\n\n#%%\ndef create_multi_feature_dataset(n_samples=2000, drift_level='mixed'):\n    \"\"\"\n    สร้าง dataset ที่มีหลาย features พร้อม simulated drift\n    \n    Parameters:\n    -----------\n    n_samples : int - จำนวน samples ต่อ dataset\n    drift_level : str - 'none', 'mild', 'severe', 'mixed'\n    \n    Returns:\n    --------\n    tuple : (reference_df, current_df, feature_info)\n    \"\"\"\n    \n    np.random.seed(42)\n    \n    # === Reference Data (Training) ===\n    reference_data = {\n        # Numerical Features\n        'age': np.random.normal(35, 10, n_samples),\n        'income': np.random.normal(50000, 15000, n_samples),\n        'credit_score': np.random.normal(650, 80, n_samples),\n        'account_balance': np.random.exponential(10000, n_samples),\n        'transaction_count': np.random.poisson(20, n_samples),\n        \n        # Categorical Features (encoded as integers)\n        'region': np.random.choice([0, 1, 2, 3], n_samples, p=[0.3, 0.3, 0.25, 0.15]),\n        'customer_type': np.random.choice([0, 1, 2], n_samples, p=[0.5, 0.35, 0.15]),\n        'product_category': np.random.choice([0, 1, 2, 3, 4], n_samples, p=[0.2, 0.2, 0.2, 0.2, 0.2])\n    }\n    \n    # === Current Data (Production) with various drift levels ===\n    if drift_level == 'none':\n        current_data = {k: v.copy() for k, v in reference_data.items()}\n        # เพิ่ม random noise เล็กน้อย\n        for key in ['age', 'income', 'credit_score']:\n            current_data[key] = np.random.normal(\n                reference_data[key].mean(),\n                reference_data[key].std(),\n                n_samples\n            )\n    \n    elif drift_level == 'mixed':\n        current_data = {\n            # No drift\n            'age': np.random.normal(35, 10, n_samples),  # เหมือนเดิม\n            'credit_score': np.random.normal(650, 80, n_samples),  # เหมือนเดิม\n            \n            # Mild drift\n            'income': np.random.normal(55000, 15000, n_samples),  # mean shift เล็กน้อย\n            'transaction_count': np.random.poisson(25, n_samples),  # เพิ่มขึ้นเล็กน้อย\n            \n            # Severe drift\n            'account_balance': np.random.exponential(20000, n_samples),  # scale เปลี่ยนมาก\n            \n            # Categorical drift\n            'region': np.random.choice([0, 1, 2, 3], n_samples, p=[0.15, 0.15, 0.35, 0.35]),  # distribution เปลี่ยน\n            'customer_type': np.random.choice([0, 1, 2], n_samples, p=[0.5, 0.35, 0.15]),  # เหมือนเดิม\n            'product_category': np.random.choice([0, 1, 2, 3, 4], n_samples, p=[0.4, 0.1, 0.1, 0.2, 0.2])  # เปลี่ยนบ้าง\n        }\n    \n    reference_df = pd.DataFrame(reference_data)\n    current_df = pd.DataFrame(current_data)\n    \n    # Feature info\n    feature_info = {\n        'numerical': ['age', 'income', 'credit_score', 'account_balance', 'transaction_count'],\n        'categorical': ['region', 'customer_type', 'product_category'],\n        'expected_drift': {\n            'age': 'none',\n            'income': 'mild',\n            'credit_score': 'none',\n            'account_balance': 'severe',\n            'transaction_count': 'mild',\n            'region': 'severe',\n            'customer_type': 'none',\n            'product_category': 'mild'\n        }\n    }\n    \n    return reference_df, current_df, feature_info\n\n# สร้าง dataset\nreference_df, current_df, feature_info = create_multi_feature_dataset(drift_level='mixed')\n\nprint(\"📊 Reference Data Shape:\", reference_df.shape)\nprint(\"📊 Current Data Shape:\", current_df.shape)\nprint(\"\\n📋 Feature Types:\")\nprint(f\"  Numerical: {feature_info['numerical']}\")\nprint(f\"  Categorical: {feature_info['categorical']}\")\n\nprint(\"\\n📋 Expected Drift Levels:\")\nfor feature, level in feature_info['expected_drift'].items():\n    print(f\"  {feature}: {level}\")\n\n#%%\n# แสดง summary statistics\nprint(\"\\n\" + \"=\" * 60)\nprint(\"REFERENCE DATA SUMMARY\")\nprint(\"=\" * 60)\nprint(reference_df.describe())\n\nprint(\"\\n\" + \"=\" * 60)\nprint(\"CURRENT DATA SUMMARY\")\nprint(\"=\" * 60)\nprint(current_df.describe())\n\n#%% [markdown]\n# ## ส่วนที่ 3: สร้าง Feature Drift Detector Class\n# \n# เราจะสร้าง class ที่รวมทุก drift detection methods เพื่อใช้งานได้สะดวก\n\n#%%\nclass FeatureDriftDetector:\n    \"\"\"\n    Class สำหรับตรวจจับ drift ในแต่ละ feature\n    \n    รองรับทั้ง numerical และ categorical features\n    ใช้หลาย statistical tests เพื่อ comprehensive analysis\n    \"\"\"\n    \n    def __init__(self, reference_data, current_data, numerical_features=None, categorical_features=None):\n        \"\"\"\n        Initialize detector\n        \n        Parameters:\n        -----------\n        reference_data : pd.DataFrame - ข้อมูล reference (training)\n        current_data : pd.DataFrame - ข้อมูล current (production)\n        numerical_features : list - รายชื่อ numerical features\n        categorical_features : list - รายชื่อ categorical features\n        \"\"\"\n        self.reference = reference_data\n        self.current = current_data\n        \n        # Auto-detect feature types if not provided\n        if numerical_features is None:\n            self.numerical_features = reference_data.select_dtypes(include=[np.number]).columns.tolist()\n        else:\n            self.numerical_features = numerical_features\n            \n        if categorical_features is None:\n            self.categorical_features = []\n        else:\n            self.categorical_features = categorical_features\n        \n        self.results = {}\n    \n    def ks_test(self, feature):\n        \"\"\"Kolmogorov-Smirnov test สำหรับ numerical features\"\"\"\n        stat, p_value = stats.ks_2samp(\n            self.reference[feature].dropna(),\n            self.current[feature].dropna()\n        )\n        return {'statistic': stat, 'p_value': p_value}\n    \n    def calculate_psi(self, feature, bins=10):\n        \"\"\"Population Stability Index\"\"\"\n        ref_data = self.reference[feature].dropna()\n        cur_data = self.current[feature].dropna()\n        \n        # สร้าง bins\n        breakpoints = np.percentile(ref_data, np.linspace(0, 100, bins + 1))\n        breakpoints = np.unique(breakpoints)\n        \n        ref_counts, _ = np.histogram(ref_data, bins=breakpoints)\n        cur_counts, _ = np.histogram(cur_data, bins=breakpoints)\n        \n        # คำนวณ proportions\n        eps = 1e-6\n        ref_props = ref_counts / len(ref_data) + eps\n        cur_props = cur_counts / len(cur_data) + eps\n        \n        # คำนวณ PSI\n        psi = np.sum((cur_props - ref_props) * np.log(cur_props / ref_props))\n        \n        return {'psi': psi}\n    \n    def wasserstein_test(self, feature):\n        \"\"\"Wasserstein Distance\"\"\"\n        ref_data = self.reference[feature].dropna()\n        cur_data = self.current[feature].dropna()\n        \n        distance = stats.wasserstein_distance(ref_data, cur_data)\n        \n        # Normalize by std of reference\n        std = ref_data.std()\n        normalized = distance / std if std > 0 else distance\n        \n        return {'distance': distance, 'normalized_distance': normalized}\n    \n    def chi_squared_test(self, feature):\n        \"\"\"Chi-squared test สำหรับ categorical features\"\"\"\n        # นับ frequency ของแต่ละ category\n        ref_counts = self.reference[feature].value_counts()\n        cur_counts = self.current[feature].value_counts()\n        \n        # รวม categories ให้เท่ากัน\n        all_categories = set(ref_counts.index) | set(cur_counts.index)\n        ref_freq = [ref_counts.get(cat, 0) for cat in all_categories]\n        cur_freq = [cur_counts.get(cat, 0) for cat in all_categories]\n        \n        # Normalize to expected frequencies\n        total_ref = sum(ref_freq)\n        total_cur = sum(cur_freq)\n        expected = [(r / total_ref) * total_cur for r in ref_freq]\n        \n        # Chi-squared test\n        try:\n            stat, p_value = stats.chisquare(cur_freq, expected)\n        except:\n            stat, p_value = 0, 1.0\n        \n        return {'statistic': stat, 'p_value': p_value}\n    \n    def analyze_numerical_feature(self, feature):\n        \"\"\"วิเคราะห์ drift สำหรับ numerical feature\"\"\"\n        results = {\n            'feature': feature,\n            'type': 'numerical',\n            'ks_test': self.ks_test(feature),\n            'psi': self.calculate_psi(feature),\n            'wasserstein': self.wasserstein_test(feature)\n        }\n        \n        # สรุปผล\n        ks_drift = results['ks_test']['p_value'] < 0.05\n        psi_value = results['psi']['psi']\n        \n        if psi_value < 0.1:\n            psi_severity = 'none'\n        elif psi_value < 0.25:\n            psi_severity = 'mild'\n        else:\n            psi_severity = 'severe'\n        \n        results['drift_detected'] = ks_drift or psi_severity != 'none'\n        results['severity'] = psi_severity\n        \n        return results\n    \n    def analyze_categorical_feature(self, feature):\n        \"\"\"วิเคราะห์ drift สำหรับ categorical feature\"\"\"\n        chi2_result = self.chi_squared_test(feature)\n        \n        # คำนวณ PSI สำหรับ categorical\n        ref_props = self.reference[feature].value_counts(normalize=True)\n        cur_props = self.current[feature].value_counts(normalize=True)\n        \n        all_cats = set(ref_props.index) | set(cur_props.index)\n        eps = 1e-6\n        psi = 0\n        for cat in all_cats:\n            ref_p = ref_props.get(cat, 0) + eps\n            cur_p = cur_props.get(cat, 0) + eps\n            psi += (cur_p - ref_p) * np.log(cur_p / ref_p)\n        \n        results = {\n            'feature': feature,\n            'type': 'categorical',\n            'chi_squared': chi2_result,\n            'psi': {'psi': psi}\n        }\n        \n        # สรุปผล\n        chi2_drift = chi2_result['p_value'] < 0.05\n        \n        if psi < 0.1:\n            psi_severity = 'none'\n        elif psi < 0.25:\n            psi_severity = 'mild'\n        else:\n            psi_severity = 'severe'\n        \n        results['drift_detected'] = chi2_drift or psi_severity != 'none'\n        results['severity'] = psi_severity\n        \n        return results\n    \n    def analyze_all_features(self):\n        \"\"\"วิเคราะห์ drift สำหรับทุก features\"\"\"\n        all_results = {}\n        \n        # Numerical features\n        for feature in self.numerical_features:\n            all_results[feature] = self.analyze_numerical_feature(feature)\n        \n        # Categorical features\n        for feature in self.categorical_features:\n            all_results[feature] = self.analyze_categorical_feature(feature)\n        \n        self.results = all_results\n        return all_results\n    \n    def get_summary_report(self):\n        \"\"\"สร้าง summary report\"\"\"\n        if not self.results:\n            self.analyze_all_features()\n        \n        summary = []\n        for feature, result in self.results.items():\n            row = {\n                'Feature': feature,\n                'Type': result['type'],\n                'Drift Detected': '✓' if result['drift_detected'] else '✗',\n                'Severity': result['severity'].upper(),\n                'PSI': f\"{result['psi']['psi']:.4f}\"\n            }\n            \n            if result['type'] == 'numerical':\n                row['KS p-value'] = f\"{result['ks_test']['p_value']:.4f}\"\n            else:\n                row['Chi2 p-value'] = f\"{result['chi_squared']['p_value']:.4f}\"\n            \n            summary.append(row)\n        \n        return pd.DataFrame(summary)\n\n#%%\n# ใช้งาน FeatureDriftDetector\ndetector = FeatureDriftDetector(\n    reference_data=reference_df,\n    current_data=current_df,\n    numerical_features=feature_info['numerical'],\n    categorical_features=feature_info['categorical']\n)\n\n# วิเคราะห์ทุก features\nall_results = detector.analyze_all_features()\n\n# แสดง summary report\nprint(\"=\" * 80)\nprint(\"📊 FEATURE DRIFT DETECTION REPORT\")\nprint(\"=\" * 80)\nsummary_df = detector.get_summary_report()\nprint(summary_df.to_string(index=False))\n\n#%% [markdown]\n# ## ส่วนที่ 4: Visualize Feature Distributions\n# \n# การ visualize ช่วยให้เข้าใจ drift ได้ดีขึ้น\n\n#%%\ndef plot_numerical_feature_drift(reference, current, feature_name, ax=None):\n    \"\"\"\n    สร้าง visualization สำหรับ numerical feature drift\n    \"\"\"\n    if ax is None:\n        fig, ax = plt.subplots(figsize=(10, 4))\n    \n    # Histogram\n    ax.hist(reference[feature_name], bins=30, alpha=0.5, label='Reference', color='blue', density=True)\n    ax.hist(current[feature_name], bins=30, alpha=0.5, label='Current', color='red', density=True)\n    \n    # Statistics\n    ref_mean = reference[feature_name].mean()\n    cur_mean = current[feature_name].mean()\n    \n    ax.axvline(ref_mean, color='blue', linestyle='--', linewidth=2)\n    ax.axvline(cur_mean, color='red', linestyle='--', linewidth=2)\n    \n    # Calculate PSI for title\n    detector_temp = FeatureDriftDetector(reference, current, [feature_name], [])\n    result = detector_temp.analyze_numerical_feature(feature_name)\n    \n    ax.set_title(f'{feature_name}\\nPSI: {result[\"psi\"][\"psi\"]:.4f} | Severity: {result[\"severity\"].upper()}')\n    ax.set_xlabel(feature_name)\n    ax.set_ylabel('Density')\n    ax.legend()\n    \n    return ax\n\ndef plot_categorical_feature_drift(reference, current, feature_name, ax=None):\n    \"\"\"\n    สร้าง visualization สำหรับ categorical feature drift\n    \"\"\"\n    if ax is None:\n        fig, ax = plt.subplots(figsize=(10, 4))\n    \n    # Calculate proportions\n    ref_props = reference[feature_name].value_counts(normalize=True).sort_index()\n    cur_props = current[feature_name].value_counts(normalize=True).sort_index()\n    \n    # Align categories\n    all_cats = sorted(set(ref_props.index) | set(cur_props.index))\n    ref_values = [ref_props.get(cat, 0) for cat in all_cats]\n    cur_values = [cur_props.get(cat, 0) for cat in all_cats]\n    \n    x = np.arange(len(all_cats))\n    width = 0.35\n    \n    ax.bar(x - width/2, ref_values, width, label='Reference', color='blue', alpha=0.7)\n    ax.bar(x + width/2, cur_values, width, label='Current', color='red', alpha=0.7)\n    \n    ax.set_xticks(x)\n    ax.set_xticklabels([f'Cat {c}' for c in all_cats])\n    \n    # Calculate PSI for title\n    detector_temp = FeatureDriftDetector(reference, current, [], [feature_name])\n    result = detector_temp.analyze_categorical_feature(feature_name)\n    \n    ax.set_title(f'{feature_name}\\nPSI: {result[\"psi\"][\"psi\"]:.4f} | Severity: {result[\"severity\"].upper()}')\n    ax.set_xlabel('Category')\n    ax.set_ylabel('Proportion')\n    ax.legend()\n    \n    return ax\n\n#%%\n# Plot all numerical features\nfig, axes = plt.subplots(2, 3, figsize=(15, 10))\naxes = axes.flatten()\n\nfor idx, feature in enumerate(feature_info['numerical']):\n    plot_numerical_feature_drift(reference_df, current_df, feature, axes[idx])\n\n# Remove empty subplot\nif len(feature_info['numerical']) < 6:\n    axes[-1].set_visible(False)\n\nplt.suptitle('Numerical Features: Distribution Comparison\\n(Reference vs Current)', fontsize=14, fontweight='bold')\nplt.tight_layout()\nplt.savefig('numerical_features_drift.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%%\n# Plot all categorical features\nfig, axes = plt.subplots(1, 3, figsize=(15, 4))\n\nfor idx, feature in enumerate(feature_info['categorical']):\n    plot_categorical_feature_drift(reference_df, current_df, feature, axes[idx])\n\nplt.suptitle('Categorical Features: Distribution Comparison\\n(Reference vs Current)', fontsize=14, fontweight='bold')\nplt.tight_layout()\nplt.savefig('categorical_features_drift.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%% [markdown]\n# ## ส่วนที่ 5: Feature Drift Ranking และ Prioritization\n# \n# จัดลำดับ features ตามความรุนแรงของ drift\n\n#%%\ndef rank_features_by_drift(detector):\n    \"\"\"\n    จัดลำดับ features ตาม drift severity\n    \"\"\"\n    if not detector.results:\n        detector.analyze_all_features()\n    \n    rankings = []\n    for feature, result in detector.results.items():\n        rankings.append({\n            'feature': feature,\n            'psi': result['psi']['psi'],\n            'drift_detected': result['drift_detected'],\n            'severity': result['severity'],\n            'type': result['type']\n        })\n    \n    # เรียงตาม PSI (มากไปน้อย)\n    rankings.sort(key=lambda x: x['psi'], reverse=True)\n    \n    return rankings\n\n# จัดลำดับ features\nrankings = rank_features_by_drift(detector)\n\nprint(\"=\" * 60)\nprint(\"📊 FEATURE DRIFT RANKING (Sorted by PSI)\")\nprint(\"=\" * 60)\n\nfor rank, item in enumerate(rankings, 1):\n    severity_emoji = {'none': '🟢', 'mild': '🟡', 'severe': '🔴'}[item['severity']]\n    print(f\"{rank}. {item['feature']:<20} | PSI: {item['psi']:.4f} | {severity_emoji} {item['severity'].upper()}\")\n\n#%%\n# สร้าง Drift Ranking Visualization\nfig, ax = plt.subplots(figsize=(12, 6))\n\nfeatures = [r['feature'] for r in rankings]\npsi_values = [r['psi'] for r in rankings]\ncolors = ['red' if r['severity'] == 'severe' else 'orange' if r['severity'] == 'mild' else 'green' for r in rankings]\n\nbars = ax.barh(features, psi_values, color=colors, alpha=0.7, edgecolor='black')\n\n# Add threshold lines\nax.axvline(0.1, color='orange', linestyle='--', label='Mild Threshold (0.1)')\nax.axvline(0.25, color='red', linestyle='--', label='Severe Threshold (0.25)')\n\nax.set_xlabel('PSI Value')\nax.set_ylabel('Feature')\nax.set_title('Feature Drift Ranking by PSI\\n(Red=Severe, Orange=Mild, Green=None)')\nax.legend()\n\n# Add value labels\nfor bar, psi in zip(bars, psi_values):\n    ax.text(psi + 0.01, bar.get_y() + bar.get_height()/2, f'{psi:.3f}', va='center')\n\nplt.tight_layout()\nplt.savefig('feature_drift_ranking.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%% [markdown]\n# ## ส่วนที่ 6: Time-based Distribution Analysis\n# \n# วิเคราะห์ว่า distribution เปลี่ยนแปลงอย่างไรเมื่อเวลาผ่านไป\n\n#%%\ndef simulate_time_series_data(n_periods=6, samples_per_period=500):\n    \"\"\"\n    จำลองข้อมูลที่เปลี่ยนแปลงตามเวลา\n    \"\"\"\n    all_data = []\n    \n    for period in range(n_periods):\n        np.random.seed(42 + period)\n        \n        # Gradual drift: mean เพิ่มขึ้นทีละน้อย\n        age_mean = 35 + period * 2  # drift ใน age\n        income_mean = 50000  # ไม่มี drift\n        \n        data = pd.DataFrame({\n            'period': period,\n            'age': np.random.normal(age_mean, 10, samples_per_period),\n            'income': np.random.normal(income_mean, 15000, samples_per_period),\n            'credit_score': np.random.normal(650, 80, samples_per_period)\n        })\n        all_data.append(data)\n    \n    return pd.concat(all_data, ignore_index=True)\n\n# สร้างข้อมูล time series\ntime_series_data = simulate_time_series_data()\nprint(f\"📊 Time Series Data Shape: {time_series_data.shape}\")\nprint(f\"📋 Periods: {time_series_data['period'].unique()}\")\n\n#%%\n# Visualize distribution over time\nfig, axes = plt.subplots(1, 3, figsize=(15, 5))\n\nfeatures_to_plot = ['age', 'income', 'credit_score']\ncolors = plt.cm.viridis(np.linspace(0, 1, 6))\n\nfor idx, feature in enumerate(features_to_plot):\n    ax = axes[idx]\n    \n    for period in range(6):\n        period_data = time_series_data[time_series_data['period'] == period][feature]\n        ax.hist(period_data, bins=20, alpha=0.3, color=colors[period], label=f'Period {period}')\n        ax.axvline(period_data.mean(), color=colors[period], linestyle='--', alpha=0.8)\n    \n    ax.set_title(f'{feature.capitalize()} Distribution Over Time')\n    ax.set_xlabel(feature)\n    ax.set_ylabel('Frequency')\n    ax.legend(fontsize=8)\n\nplt.suptitle('Feature Distributions Across Time Periods\\n(Dashed lines = Mean per period)', fontsize=12, fontweight='bold')\nplt.tight_layout()\nplt.savefig('time_series_distributions.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%%\n# Calculate PSI over time (comparing each period to Period 0)\ndef calculate_psi_over_time(data, feature, reference_period=0):\n    \"\"\"\n    คำนวณ PSI เมื่อเทียบกับ reference period\n    \"\"\"\n    ref_data = data[data['period'] == reference_period][feature]\n    \n    results = []\n    for period in data['period'].unique():\n        if period == reference_period:\n            results.append({'period': period, 'psi': 0})\n        else:\n            cur_data = data[data['period'] == period][feature]\n            \n            # Calculate PSI\n            breakpoints = np.percentile(ref_data, np.linspace(0, 100, 11))\n            breakpoints = np.unique(breakpoints)\n            \n            ref_counts, _ = np.histogram(ref_data, bins=breakpoints)\n            cur_counts, _ = np.histogram(cur_data, bins=breakpoints)\n            \n            eps = 1e-6\n            ref_props = ref_counts / len(ref_data) + eps\n            cur_props = cur_counts / len(cur_data) + eps\n            \n            psi = np.sum((cur_props - ref_props) * np.log(cur_props / ref_props))\n            results.append({'period': period, 'psi': psi})\n    \n    return pd.DataFrame(results)\n\n# คำนวณ PSI สำหรับแต่ละ feature\nfig, ax = plt.subplots(figsize=(10, 6))\n\nfor feature in ['age', 'income', 'credit_score']:\n    psi_over_time = calculate_psi_over_time(time_series_data, feature)\n    ax.plot(psi_over_time['period'], psi_over_time['psi'], marker='o', label=feature, linewidth=2)\n\nax.axhline(0.1, color='orange', linestyle='--', alpha=0.7, label='Mild Threshold')\nax.axhline(0.25, color='red', linestyle='--', alpha=0.7, label='Severe Threshold')\n\nax.set_xlabel('Time Period')\nax.set_ylabel('PSI (compared to Period 0)')\nax.set_title('PSI Trend Over Time\\n(Reference: Period 0)')\nax.legend()\nax.grid(True, alpha=0.3)\n\nplt.tight_layout()\nplt.savefig('psi_over_time.png', dpi=150, bbox_inches='tight')\nplt.show()\n\nprint(\"\\n💡 สังเกต: Age แสดง gradual drift (PSI เพิ่มขึ้นเรื่อยๆ)\")\nprint(\"   ในขณะที่ Income และ Credit Score ค่อนข้าง stable\")\n\n#%% [markdown]\n# ## สรุป LAB 2\n# \n# ### สิ่งที่เรียนรู้:\n# 1. **FeatureDriftDetector Class**: เครื่องมือสำหรับวิเคราะห์ drift ทุก features\n# 2. **Numerical vs Categorical**: ใช้ methods ที่เหมาะสมกับแต่ละ type\n# 3. **Visualization**: ช่วยให้เห็น drift ได้ชัดเจน\n# 4. **Ranking**: จัดลำดับ features ตาม severity เพื่อ prioritization\n# 5. **Time-based Analysis**: ติดตาม drift เมื่อเวลาผ่านไป\n\n#%%\nprint(\"=\" * 60)\nprint(\"✅ LAB 2 COMPLETED!\")\nprint(\"=\" * 60)\nprint(\"\"\"\n📚 Key Takeaways:\n1. Numerical features: ใช้ KS Test + PSI + Wasserstein\n2. Categorical features: ใช้ Chi-squared + PSI\n3. Visualization สำคัญสำหรับการทำความเข้าใจ drift\n4. Track PSI over time เพื่อ detect gradual drift\n\n🔜 Next: LAB 3 - Multivariate Drift Analysis\n\"\"\")\n```\n\n---\n\n## LAB 3: Multivariate Drift Analysis\n\n```python\n#%% [markdown]\n# # LAB 3: Multivariate Drift Analysis\n# ## การวิเคราะห์ Drift ที่เกิดจากความสัมพันธ์ระหว่าง Features\n# \n# ### วัตถุประสงค์การเรียนรู้:\n# 1. ตรวจจับ drift ที่เกิดจากความสัมพันธ์ระหว่าง features\n# 2. ใช้ Dataset-level drift detection\n# 3. วิเคราะห์ Correlation changes ระหว่าง features\n#\n# ### ทฤษฎี:\n# **Multivariate Drift** เกิดขึ้นเมื่อ:\n# - แต่ละ feature ดูปกติเมื่อวิเคราะห์แยก\n# - แต่ความสัมพันธ์ระหว่าง features เปลี่ยนไป\n# - ตัวอย่าง: correlation ระหว่าง age และ income เปลี่ยน\n\n#%% [markdown]\n# ## ส่วนที่ 1: เตรียม Environment\n\n#%%\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom scipy import stats\nfrom sklearn.decomposition import PCA\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.covariance import EmpiricalCovariance\nimport warnings\nwarnings.filterwarnings('ignore')\n\nnp.random.seed(42)\nplt.rcParams['figure.figsize'] = (12, 8)\n\nprint(\"✅ Libraries imported successfully!\")\n\n#%% [markdown]\n# ## ส่วนที่ 2: สร้าง Dataset ที่มี Multivariate Drift\n# \n# เราจะสร้างข้อมูลที่:\n# - Marginal distributions เหมือนกัน (ไม่มี univariate drift)\n# - แต่ correlation structure เปลี่ยน (multivariate drift)\n\n#%%\ndef create_multivariate_drift_data(n_samples=2000):\n    \"\"\"\n    สร้างข้อมูลที่มี multivariate drift\n    - Marginal distributions คล้ายกัน\n    - Correlation structure ต่างกัน\n    \"\"\"\n    np.random.seed(42)\n    \n    # === Reference Data ===\n    # Correlated features: age และ income มี positive correlation สูง\n    mean_ref = [35, 50000, 650]  # age, income, credit_score\n    cov_ref = [\n        [100, 3000, 50],      # age variance และ covariance\n        [3000, 225000000, 100000],  # income variance และ covariance\n        [50, 100000, 6400]    # credit_score variance และ covariance\n    ]\n    \n    ref_data = np.random.multivariate_normal(mean_ref, cov_ref, n_samples)\n    reference_df = pd.DataFrame(ref_data, columns=['age', 'income', 'credit_score'])\n    \n    # === Current Data (Multivariate Drift) ===\n    # Same marginals but different correlation structure\n    mean_cur = [35, 50000, 650]  # means เหมือนเดิม\n    \n    # Correlation structure เปลี่ยน: age และ income correlation ลดลง\n    cov_cur = [\n        [100, 500, 50],       # correlation ระหว่าง age-income ลดลงมาก!\n        [500, 225000000, 100000],\n        [50, 100000, 6400]\n    ]\n    \n    cur_data = np.random.multivariate_normal(mean_cur, cov_cur, n_samples)\n    current_df = pd.DataFrame(cur_data, columns=['age', 'income', 'credit_score'])\n    \n    return reference_df, current_df\n\n# สร้างข้อมูล\nref_multi, cur_multi = create_multivariate_drift_data()\n\nprint(\"📊 Reference Data:\")\nprint(ref_multi.describe())\nprint(\"\\n📊 Current Data:\")\nprint(cur_multi.describe())\n\n#%%\n# แสดง correlation matrix comparison\nfig, axes = plt.subplots(1, 3, figsize=(15, 4))\n\n# Reference correlation matrix\ncorr_ref = ref_multi.corr()\nsns.heatmap(corr_ref, annot=True, cmap='coolwarm', center=0, ax=axes[0], \n            vmin=-1, vmax=1, fmt='.3f')\naxes[0].set_title('Reference Correlation Matrix')\n\n# Current correlation matrix\ncorr_cur = cur_multi.corr()\nsns.heatmap(corr_cur, annot=True, cmap='coolwarm', center=0, ax=axes[1],\n            vmin=-1, vmax=1, fmt='.3f')\naxes[1].set_title('Current Correlation Matrix')\n\n# Difference\ncorr_diff = corr_cur - corr_ref\nsns.heatmap(corr_diff, annot=True, cmap='RdBu', center=0, ax=axes[2],\n            vmin=-1, vmax=1, fmt='.3f')\naxes[2].set_title('Correlation Difference\\n(Current - Reference)')\n\nplt.tight_layout()\nplt.savefig('correlation_comparison.png', dpi=150, bbox_inches='tight')\nplt.show()\n\nprint(\"\\n💡 สังเกต: Correlation ระหว่าง Age-Income เปลี่ยนจาก ~0.6 เป็น ~0.03\")\n\n#%% [markdown]\n# ## ส่วนที่ 3: Univariate vs Multivariate Drift Detection\n# \n# เปรียบเทียบว่า univariate methods พลาดอะไรไป\n\n#%%\nfrom scipy import stats\n\ndef univariate_drift_check(ref_df, cur_df):\n    \"\"\"ตรวจจับ drift ด้วย univariate methods\"\"\"\n    results = []\n    \n    for col in ref_df.columns:\n        # KS Test\n        ks_stat, ks_pval = stats.ks_2samp(ref_df[col], cur_df[col])\n        \n        # Mean comparison\n        ref_mean = ref_df[col].mean()\n        cur_mean = cur_df[col].mean()\n        mean_diff_pct = abs(cur_mean - ref_mean) / ref_mean * 100\n        \n        # Std comparison\n        ref_std = ref_df[col].std()\n        cur_std = cur_df[col].std()\n        std_diff_pct = abs(cur_std - ref_std) / ref_std * 100\n        \n        results.append({\n            'feature': col,\n            'ks_statistic': ks_stat,\n            'ks_pvalue': ks_pval,\n            'drift_detected': ks_pval < 0.05,\n            'mean_diff_%': mean_diff_pct,\n            'std_diff_%': std_diff_pct\n        })\n    \n    return pd.DataFrame(results)\n\n# ตรวจจับ univariate drift\nunivariate_results = univariate_drift_check(ref_multi, cur_multi)\nprint(\"=\" * 60)\nprint(\"📊 UNIVARIATE DRIFT DETECTION RESULTS\")\nprint(\"=\" * 60)\nprint(univariate_results.to_string(index=False))\n\nprint(\"\\n💡 สังเกต: Univariate methods ไม่พบ drift ที่สำคัญ!\")\nprint(\"   แต่เรารู้ว่า correlation structure เปลี่ยนไป\")\n\n#%% [markdown]\n# ## ส่วนที่ 4: Correlation-based Drift Detection\n# \n# ตรวจจับการเปลี่ยนแปลงของ correlation structure\n\n#%%\ndef correlation_drift_test(ref_df, cur_df, significance_level=0.05):\n    \"\"\"\n    ตรวจจับ drift ใน correlation structure\n    \n    ใช้ Fisher's Z transformation เพื่อเปรียบเทียบ correlations\n    \"\"\"\n    results = []\n    \n    # คำนวณ correlation matrices\n    ref_corr = ref_df.corr()\n    cur_corr = cur_df.corr()\n    \n    n_ref = len(ref_df)\n    n_cur = len(cur_df)\n    \n    # เปรียบเทียบแต่ละ pair\n    for i, col1 in enumerate(ref_df.columns):\n        for j, col2 in enumerate(ref_df.columns):\n            if i >= j:  # ข้าม diagonal และ lower triangle\n                continue\n            \n            r_ref = ref_corr.loc[col1, col2]\n            r_cur = cur_corr.loc[col1, col2]\n            \n            # Fisher's Z transformation\n            z_ref = np.arctanh(r_ref)\n            z_cur = np.arctanh(r_cur)\n            \n            # Standard error\n            se = np.sqrt(1/(n_ref-3) + 1/(n_cur-3))\n            \n            # Z-test statistic\n            z_stat = (z_ref - z_cur) / se\n            p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))\n            \n            results.append({\n                'feature_pair': f'{col1} - {col2}',\n                'ref_correlation': r_ref,\n                'cur_correlation': r_cur,\n                'correlation_change': r_cur - r_ref,\n                'z_statistic': z_stat,\n                'p_value': p_value,\n                'significant_change': p_value < significance_level\n            })\n    \n    return pd.DataFrame(results)\n\n# ตรวจจับ correlation drift\ncorr_drift_results = correlation_drift_test(ref_multi, cur_multi)\nprint(\"=\" * 60)\nprint(\"📊 CORRELATION DRIFT DETECTION RESULTS\")\nprint(\"=\" * 60)\nprint(corr_drift_results.to_string(index=False))\n\n#%%\n# Visualize correlation changes\nfig, ax = plt.subplots(figsize=(10, 6))\n\nx = range(len(corr_drift_results))\ncolors = ['red' if sig else 'green' for sig in corr_drift_results['significant_change']]\n\nbars = ax.bar(x, corr_drift_results['correlation_change'], color=colors, alpha=0.7, edgecolor='black')\n\nax.set_xticks(x)\nax.set_xticklabels(corr_drift_results['feature_pair'], rotation=45, ha='right')\nax.set_ylabel('Correlation Change')\nax.set_title('Correlation Changes Between Reference and Current Data\\n(Red = Statistically Significant)')\nax.axhline(0, color='black', linestyle='-', linewidth=0.5)\n\n# Add value labels\nfor bar, val in zip(bars, corr_drift_results['correlation_change']):\n    height = bar.get_height()\n    ax.text(bar.get_x() + bar.get_width()/2., height, f'{val:.3f}',\n            ha='center', va='bottom' if height > 0 else 'top')\n\nplt.tight_layout()\nplt.savefig('correlation_drift.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%% [markdown]\n# ## ส่วนที่ 5: PCA-based Multivariate Drift Detection\n# \n# ใช้ PCA เพื่อตรวจจับ drift ใน multivariate structure\n\n#%%\ndef pca_drift_detection(ref_df, cur_df, n_components=None):\n    \"\"\"\n    ใช้ PCA เพื่อตรวจจับ multivariate drift\n    \n    เปรียบเทียบ:\n    1. Explained variance ratios\n    2. Principal component directions\n    3. Reconstruction errors\n    \"\"\"\n    if n_components is None:\n        n_components = min(len(ref_df.columns), 3)\n    \n    # Standardize data\n    scaler = StandardScaler()\n    ref_scaled = scaler.fit_transform(ref_df)\n    cur_scaled = scaler.transform(cur_df)\n    \n    # Fit PCA on reference data\n    pca_ref = PCA(n_components=n_components)\n    pca_ref.fit(ref_scaled)\n    \n    # Transform both datasets using reference PCA\n    ref_transformed = pca_ref.transform(ref_scaled)\n    cur_transformed = pca_ref.transform(cur_scaled)\n    \n    # 1. Compare explained variance\n    ref_explained_var = pca_ref.explained_variance_ratio_\n    \n    # Fit PCA on current data for comparison\n    pca_cur = PCA(n_components=n_components)\n    pca_cur.fit(cur_scaled)\n    cur_explained_var = pca_cur.explained_variance_ratio_\n    \n    # 2. Compare principal components (using cosine similarity)\n    component_similarities = []\n    for i in range(n_components):\n        cos_sim = np.dot(pca_ref.components_[i], pca_cur.components_[i])\n        cos_sim = abs(cos_sim)  # Absolute value เพราะ sign อาจ flip\n        component_similarities.append(cos_sim)\n    \n    # 3. Reconstruction error on current data using reference PCA\n    ref_reconstructed = pca_ref.inverse_transform(ref_transformed)\n    cur_reconstructed = pca_ref.inverse_transform(cur_transformed)\n    \n    ref_recon_error = np.mean((ref_scaled - ref_reconstructed) ** 2)\n    cur_recon_error = np.mean((cur_scaled - cur_reconstructed) ** 2)\n    \n    results = {\n        'ref_explained_variance': ref_explained_var,\n        'cur_explained_variance': cur_explained_var,\n        'component_similarities': component_similarities,\n        'ref_reconstruction_error': ref_recon_error,\n        'cur_reconstruction_error': cur_recon_error,\n        'reconstruction_error_ratio': cur_recon_error / ref_recon_error if ref_recon_error > 0 else 1,\n        'ref_components': pca_ref.components_,\n        'cur_components': pca_cur.components_,\n        'pca_ref': pca_ref,\n        'ref_transformed': ref_transformed,\n        'cur_transformed': cur_transformed\n    }\n    \n    return results\n\n# ทำ PCA drift detection\npca_results = pca_drift_detection(ref_multi, cur_multi)\n\nprint(\"=\" * 60)\nprint(\"📊 PCA-BASED DRIFT DETECTION RESULTS\")\nprint(\"=\" * 60)\n\nprint(\"\\n1. Explained Variance Comparison:\")\nfor i in range(len(pca_results['ref_explained_variance'])):\n    ref_ev = pca_results['ref_explained_variance'][i]\n    cur_ev = pca_results['cur_explained_variance'][i]\n    print(f\"   PC{i+1}: Reference = {ref_ev:.4f}, Current = {cur_ev:.4f}, Diff = {cur_ev - ref_ev:.4f}\")\n\nprint(\"\\n2. Component Similarities (1.0 = identical):\")\nfor i, sim in enumerate(pca_results['component_similarities']):\n    status = \"✓ Similar\" if sim > 0.9 else \"⚠️ Changed!\"\n    print(f\"   PC{i+1}: Cosine Similarity = {sim:.4f} {status}\")\n\nprint(\"\\n3. Reconstruction Error:\")\nprint(f\"   Reference: {pca_results['ref_reconstruction_error']:.6f}\")\nprint(f\"   Current: {pca_results['cur_reconstruction_error']:.6f}\")\nprint(f\"   Ratio: {pca_results['reconstruction_error_ratio']:.4f}x\")\n\n#%%\n# Visualize PCA results\nfig, axes = plt.subplots(2, 2, figsize=(14, 10))\n\n# Plot 1: Explained variance comparison\nax1 = axes[0, 0]\nx = range(1, len(pca_results['ref_explained_variance']) + 1)\nwidth = 0.35\nax1.bar([i - width/2 for i in x], pca_results['ref_explained_variance'], \n        width, label='Reference', color='blue', alpha=0.7)\nax1.bar([i + width/2 for i in x], pca_results['cur_explained_variance'], \n        width, label='Current', color='red', alpha=0.7)\nax1.set_xlabel('Principal Component')\nax1.set_ylabel('Explained Variance Ratio')\nax1.set_title('Explained Variance Comparison')\nax1.legend()\nax1.set_xticks(x)\n\n# Plot 2: Component similarities\nax2 = axes[0, 1]\ncolors = ['green' if s > 0.9 else 'red' for s in pca_results['component_similarities']]\nax2.bar(x, pca_results['component_similarities'], color=colors, alpha=0.7, edgecolor='black')\nax2.axhline(0.9, color='orange', linestyle='--', label='Similarity Threshold (0.9)')\nax2.set_xlabel('Principal Component')\nax2.set_ylabel('Cosine Similarity')\nax2.set_title('Principal Component Similarity\\n(Reference vs Current)')\nax2.set_xticks(x)\nax2.legend()\n\n# Plot 3: Scatter plot in PC space (Reference)\nax3 = axes[1, 0]\nax3.scatter(pca_results['ref_transformed'][:, 0], pca_results['ref_transformed'][:, 1], \n            alpha=0.3, label='Reference', color='blue')\nax3.scatter(pca_results['cur_transformed'][:, 0], pca_results['cur_transformed'][:, 1], \n            alpha=0.3, label='Current', color='red')\nax3.set_xlabel('PC1')\nax3.set_ylabel('PC2')\nax3.set_title('Data in PCA Space\\n(Using Reference PCA)')\nax3.legend()\n\n# Plot 4: Component loadings comparison\nax4 = axes[1, 1]\nfeatures = ref_multi.columns.tolist()\nx = np.arange(len(features))\nwidth = 0.35\n\nfor i in range(2):  # แสดง PC1 และ PC2\n    ref_loadings = pca_results['ref_components'][i]\n    cur_loadings = pca_results['cur_components'][i]\n    \n    ax4.barh([f'PC{i+1} Ref' for f in features] if i == 0 else [f'PC{i+1} Cur' for f in features], \n             ref_loadings if i == 0 else cur_loadings)\n\nax4.set_xlabel('Loading')\nax4.set_title('PC Loadings (First 2 Components)')\n\nplt.tight_layout()\nplt.savefig('pca_drift_analysis.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%% [markdown]\n# ## ส่วนที่ 6: Mahalanobis Distance for Dataset-level Drift\n# \n# ใช้ Mahalanobis distance เพื่อวัด multivariate drift\n\n#%%\ndef mahalanobis_drift_detection(ref_df, cur_df, threshold_percentile=95):\n    \"\"\"\n    ใช้ Mahalanobis distance เพื่อตรวจจับ multivariate drift\n    \n    วัดว่าข้อมูลใหม่อยู่ห่างจาก distribution ของ reference data เท่าไร\n    \"\"\"\n    # Standardize features\n    scaler = StandardScaler()\n    ref_scaled = scaler.fit_transform(ref_df)\n    cur_scaled = scaler.transform(cur_df)\n    \n    # Fit covariance on reference data\n    cov = EmpiricalCovariance().fit(ref_scaled)\n    \n    # Calculate Mahalanobis distances\n    ref_distances = cov.mahalanobis(ref_scaled)\n    cur_distances = cov.mahalanobis(cur_scaled)\n    \n    # Determine threshold from reference data\n    threshold = np.percentile(ref_distances, threshold_percentile)\n    \n    # Count outliers\n    ref_outliers = np.sum(ref_distances > threshold) / len(ref_distances) * 100\n    cur_outliers = np.sum(cur_distances > threshold) / len(cur_distances) * 100\n    \n    # Statistical comparison\n    ks_stat, ks_pval = stats.ks_2samp(ref_distances, cur_distances)\n    \n    results = {\n        'ref_distances': ref_distances,\n        'cur_distances': cur_distances,\n        'threshold': threshold,\n        'ref_mean_distance': np.mean(ref_distances),\n        'cur_mean_distance': np.mean(cur_distances),\n        'ref_outlier_pct': ref_outliers,\n        'cur_outlier_pct': cur_outliers,\n        'ks_statistic': ks_stat,\n        'ks_pvalue': ks_pval,\n        'drift_detected': ks_pval < 0.05\n    }\n    \n    return results\n\n# ทำ Mahalanobis drift detection\nmaha_results = mahalanobis_drift_detection(ref_multi, cur_multi)\n\nprint(\"=\" * 60)\nprint(\"📊 MAHALANOBIS DISTANCE DRIFT DETECTION\")\nprint(\"=\" * 60)\nprint(f\"\\nMean Mahalanobis Distance:\")\nprint(f\"  Reference: {maha_results['ref_mean_distance']:.4f}\")\nprint(f\"  Current: {maha_results['cur_mean_distance']:.4f}\")\nprint(f\"\\nOutlier Percentage (beyond {maha_results['threshold']:.2f} threshold):\")\nprint(f\"  Reference: {maha_results['ref_outlier_pct']:.2f}%\")\nprint(f\"  Current: {maha_results['cur_outlier_pct']:.2f}%\")\nprint(f\"\\nKS Test on Distances:\")\nprint(f\"  Statistic: {maha_results['ks_statistic']:.4f}\")\nprint(f\"  P-value: {maha_results['ks_pvalue']:.4f}\")\nprint(f\"  Drift Detected: {'Yes ✓' if maha_results['drift_detected'] else 'No'}\")\n\n#%%\n# Visualize Mahalanobis distances\nfig, axes = plt.subplots(1, 2, figsize=(14, 5))\n\n# Plot 1: Distribution of Mahalanobis distances\nax1 = axes[0]\nax1.hist(maha_results['ref_distances'], bins=50, alpha=0.5, label='Reference', color='blue', density=True)\nax1.hist(maha_results['cur_distances'], bins=50, alpha=0.5, label='Current', color='red', density=True)\nax1.axvline(maha_results['threshold'], color='orange', linestyle='--', linewidth=2, \n            label=f'95th percentile threshold ({maha_results[\"threshold\"]:.2f})')\nax1.set_xlabel('Mahalanobis Distance')\nax1.set_ylabel('Density')\nax1.set_title('Distribution of Mahalanobis Distances')\nax1.legend()\n\n# Plot 2: CDF comparison\nax2 = axes[1]\nsorted_ref = np.sort(maha_results['ref_distances'])\nsorted_cur = np.sort(maha_results['cur_distances'])\n\nax2.plot(sorted_ref, np.arange(1, len(sorted_ref)+1)/len(sorted_ref), \n         'b-', label='Reference CDF', linewidth=2)\nax2.plot(sorted_cur, np.arange(1, len(sorted_cur)+1)/len(sorted_cur), \n         'r-', label='Current CDF', linewidth=2)\nax2.axvline(maha_results['threshold'], color='orange', linestyle='--', linewidth=2)\nax2.set_xlabel('Mahalanobis Distance')\nax2.set_ylabel('Cumulative Probability')\nax2.set_title(f'CDF Comparison\\nKS Statistic = {maha_results[\"ks_statistic\"]:.4f}')\nax2.legend()\n\nplt.tight_layout()\nplt.savefig('mahalanobis_drift.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%% [markdown]\n# ## ส่วนที่ 7: Comprehensive Multivariate Drift Report\n\n#%%\nclass MultivariateriftDetector:\n    \"\"\"\n    Class สำหรับ comprehensive multivariate drift detection\n    \"\"\"\n    \n    def __init__(self, reference_df, current_df):\n        self.reference = reference_df\n        self.current = current_df\n        self.results = {}\n    \n    def analyze(self):\n        \"\"\"ทำ comprehensive analysis\"\"\"\n        \n        # 1. Correlation analysis\n        corr_results = correlation_drift_test(self.reference, self.current)\n        self.results['correlation'] = corr_results\n        \n        # 2. PCA analysis\n        pca_results = pca_drift_detection(self.reference, self.current)\n        self.results['pca'] = pca_results\n        \n        # 3. Mahalanobis analysis\n        maha_results = mahalanobis_drift_detection(self.reference, self.current)\n        self.results['mahalanobis'] = maha_results\n        \n        # 4. Overall assessment\n        drift_indicators = {\n            'correlation_drift': any(corr_results['significant_change']),\n            'pca_structure_change': any(sim < 0.9 for sim in pca_results['component_similarities']),\n            'mahalanobis_drift': maha_results['drift_detected']\n        }\n        \n        drift_count = sum(drift_indicators.values())\n        \n        self.results['summary'] = {\n            'drift_indicators': drift_indicators,\n            'drift_count': drift_count,\n            'overall_assessment': 'HIGH' if drift_count >= 2 else 'MEDIUM' if drift_count == 1 else 'LOW'\n        }\n        \n        return self.results\n    \n    def print_report(self):\n        \"\"\"พิมพ์ report สรุป\"\"\"\n        if not self.results:\n            self.analyze()\n        \n        print(\"=\" * 70)\n        print(\"📊 COMPREHENSIVE MULTIVARIATE DRIFT REPORT\")\n        print(\"=\" * 70)\n        \n        print(\"\\n1️⃣ CORRELATION DRIFT:\")\n        print(\"-\" * 40)\n        corr_df = self.results['correlation']\n        significant_pairs = corr_df[corr_df['significant_change']]\n        if len(significant_pairs) > 0:\n            print(\"   ⚠️ Significant correlation changes detected:\")\n            for _, row in significant_pairs.iterrows():\n                print(f\"      {row['feature_pair']}: {row['ref_correlation']:.3f} → {row['cur_correlation']:.3f}\")\n        else:\n            print(\"   ✓ No significant correlation changes\")\n        \n        print(\"\\n2️⃣ PCA STRUCTURE ANALYSIS:\")\n        print(\"-\" * 40)\n        pca_res = self.results['pca']\n        for i, sim in enumerate(pca_res['component_similarities']):\n            status = \"✓\" if sim > 0.9 else \"⚠️ Changed\"\n            print(f\"   PC{i+1} similarity: {sim:.4f} {status}\")\n        print(f\"   Reconstruction error ratio: {pca_res['reconstruction_error_ratio']:.4f}x\")\n        \n        print(\"\\n3️⃣ MAHALANOBIS DISTANCE ANALYSIS:\")\n        print(\"-\" * 40)\n        maha_res = self.results['mahalanobis']\n        print(f\"   Mean distance ratio: {maha_res['cur_mean_distance']/maha_res['ref_mean_distance']:.4f}x\")\n        print(f\"   Outlier % (Reference): {maha_res['ref_outlier_pct']:.2f}%\")\n        print(f\"   Outlier % (Current): {maha_res['cur_outlier_pct']:.2f}%\")\n        print(f\"   KS Test p-value: {maha_res['ks_pvalue']:.4f}\")\n        \n        print(\"\\n\" + \"=\" * 70)\n        print(\"📋 OVERALL ASSESSMENT\")\n        print(\"=\" * 70)\n        summary = self.results['summary']\n        print(f\"   Drift Indicators Triggered: {summary['drift_count']}/3\")\n        for indicator, triggered in summary['drift_indicators'].items():\n            status = \"⚠️\" if triggered else \"✓\"\n            print(f\"      {status} {indicator}: {'Yes' if triggered else 'No'}\")\n        \n        severity_emoji = {'LOW': '🟢', 'MEDIUM': '🟡', 'HIGH': '🔴'}\n        print(f\"\\n   🎯 OVERALL MULTIVARIATE DRIFT: {severity_emoji[summary['overall_assessment']]} {summary['overall_assessment']}\")\n\n# ใช้งาน\ndetector = MultivariateriftDetector(ref_multi, cur_multi)\ndetector.analyze()\ndetector.print_report()\n\n#%% [markdown]\n# ## สรุป LAB 3\n# \n# ### สิ่งที่เรียนรู้:\n# 1. **Multivariate Drift**: เกิดขึ้นเมื่อ relationship ระหว่าง features เปลี่ยน\n# 2. **Correlation Analysis**: ตรวจจับการเปลี่ยนแปลงของ pairwise correlations\n# 3. **PCA Analysis**: ตรวจจับการเปลี่ยนแปลงของ multivariate structure\n# 4. **Mahalanobis Distance**: วัด dataset-level drift\n# \n# ### Key Insights:\n# - Univariate methods อาจพลาด multivariate drift\n# - ใช้หลาย methods ร่วมกันเพื่อความครอบคลุม\n# - Monitor ทั้ง individual features และ relationships\n\n#%%\nprint(\"=\" * 60)\nprint(\"✅ LAB 3 COMPLETED!\")\nprint(\"=\" * 60)\nprint(\"\"\"\n📚 Key Takeaways:\n1. Multivariate drift ตรวจจับด้วย univariate methods ไม่ได้\n2. Correlation drift analysis ใช้ Fisher's Z transformation\n3. PCA structure comparison ดู component similarities\n4. Mahalanobis distance วัด overall distribution shift\n\n🔜 Next: LAB 4 - Drift Detection in Production Simulation\n\"\"\")\n```\n\n---\n\n## LAB 4: Drift Detection in Production Simulation\n\n```python\n#%% [markdown]\n# # LAB 4: Drift Detection in Production Simulation\n# ## การจำลองการตรวจจับ Drift ใน Production Environment\n# \n# ### วัตถุประสงค์การเรียนรู้:\n# 1. สร้าง simulated data stream ที่มี gradual drift\n# 2. ตรวจจับ sudden vs gradual drift\n# 3. Implement sliding window monitoring\n#\n# ### ทฤษฎี:\n# ใน production environment:\n# - ข้อมูลมาเป็น stream ไม่ใช่ batch\n# - Drift อาจเกิดแบบ sudden หรือ gradual\n# - ต้องมี monitoring strategy ที่เหมาะสม\n\n#%% [markdown]\n# ## ส่วนที่ 1: เตรียม Environment\n\n#%%\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom scipy import stats\nfrom collections import deque\nfrom datetime import datetime, timedelta\nimport warnings\nwarnings.filterwarnings('ignore')\n\nnp.random.seed(42)\nplt.rcParams['figure.figsize'] = (14, 6)\n\nprint(\"✅ Libraries imported successfully!\")\n\n#%% [markdown]\n# ## ส่วนที่ 2: สร้าง Data Stream Simulator\n# \n# จำลอง data stream ที่มี drift หลายรูปแบบ\n\n#%%\nclass DataStreamSimulator:\n    \"\"\"\n    Simulator สำหรับสร้าง data stream ที่มี drift patterns ต่างๆ\n    \n    Drift Types:\n    - sudden: เปลี่ยนทันทีที่จุดใดจุดหนึ่ง\n    - gradual: เปลี่ยนแปลงช้าๆ ตามเวลา\n    - incremental: เปลี่ยนเป็นขั้นบันได\n    - seasonal: เปลี่ยนตาม pattern ซ้ำ\n    - no_drift: ไม่มี drift\n    \"\"\"\n    \n    def __init__(self, base_mean=50, base_std=10, random_seed=42):\n        self.base_mean = base_mean\n        self.base_std = base_std\n        self.random_seed = random_seed\n        np.random.seed(random_seed)\n    \n    def generate_stream(self, n_samples, drift_type='no_drift', drift_params=None):\n        \"\"\"\n        สร้าง data stream\n        \n        Parameters:\n        -----------\n        n_samples : int - จำนวน samples\n        drift_type : str - ประเภทของ drift\n        drift_params : dict - parameters สำหรับ drift\n        \"\"\"\n        if drift_params is None:\n            drift_params = {}\n        \n        data = np.zeros(n_samples)\n        timestamps = [datetime(2024, 1, 1) + timedelta(hours=i) for i in range(n_samples)]\n        \n        if drift_type == 'no_drift':\n            data = np.random.normal(self.base_mean, self.base_std, n_samples)\n        \n        elif drift_type == 'sudden':\n            drift_point = drift_params.get('drift_point', n_samples // 2)\n            new_mean = drift_params.get('new_mean', self.base_mean + 2 * self.base_std)\n            \n            data[:drift_point] = np.random.normal(self.base_mean, self.base_std, drift_point)\n            data[drift_point:] = np.random.normal(new_mean, self.base_std, n_samples - drift_point)\n        \n        elif drift_type == 'gradual':\n            drift_start = drift_params.get('drift_start', n_samples // 4)\n            drift_end = drift_params.get('drift_end', 3 * n_samples // 4)\n            final_mean = drift_params.get('final_mean', self.base_mean + 2 * self.base_std)\n            \n            for i in range(n_samples):\n                if i < drift_start:\n                    current_mean = self.base_mean\n                elif i > drift_end:\n                    current_mean = final_mean\n                else:\n                    # Linear interpolation\n                    progress = (i - drift_start) / (drift_end - drift_start)\n                    current_mean = self.base_mean + progress * (final_mean - self.base_mean)\n                \n                data[i] = np.random.normal(current_mean, self.base_std)\n        \n        elif drift_type == 'incremental':\n            step_size = drift_params.get('step_size', n_samples // 5)\n            step_increase = drift_params.get('step_increase', self.base_std * 0.5)\n            \n            for i in range(n_samples):\n                step = i // step_size\n                current_mean = self.base_mean + step * step_increase\n                data[i] = np.random.normal(current_mean, self.base_std)\n        \n        elif drift_type == 'seasonal':\n            period = drift_params.get('period', 168)  # 1 week in hours\n            amplitude = drift_params.get('amplitude', self.base_std)\n            \n            for i in range(n_samples):\n                seasonal_effect = amplitude * np.sin(2 * np.pi * i / period)\n                data[i] = np.random.normal(self.base_mean + seasonal_effect, self.base_std)\n        \n        return pd.DataFrame({\n            'timestamp': timestamps,\n            'value': data,\n            'index': range(n_samples)\n        })\n\n# สร้าง simulator\nsimulator = DataStreamSimulator(base_mean=50, base_std=10)\n\nprint(\"✅ DataStreamSimulator created\")\n\n#%%\n# สร้างตัวอย่าง drift patterns\nfig, axes = plt.subplots(2, 3, figsize=(15, 8))\naxes = axes.flatten()\n\ndrift_types = ['no_drift', 'sudden', 'gradual', 'incremental', 'seasonal']\ndrift_params_list = [\n    {},\n    {'drift_point': 500, 'new_mean': 70},\n    {'drift_start': 200, 'drift_end': 800, 'final_mean': 70},\n    {'step_size': 200, 'step_increase': 5},\n    {'period': 200, 'amplitude': 15}\n]\n\nstreams = {}\nfor i, (dtype, params) in enumerate(zip(drift_types, drift_params_list)):\n    stream = simulator.generate_stream(1000, drift_type=dtype, drift_params=params)\n    streams[dtype] = stream\n    \n    ax = axes[i]\n    ax.plot(stream['index'], stream['value'], alpha=0.7, linewidth=0.5)\n    \n    # Add rolling mean\n    rolling_mean = stream['value'].rolling(window=50).mean()\n    ax.plot(stream['index'], rolling_mean, 'r-', linewidth=2, label='Rolling Mean (50)')\n    \n    ax.axhline(50, color='green', linestyle='--', alpha=0.5, label='Original Mean')\n    ax.set_title(f'{dtype.replace(\"_\", \" \").title()}')\n    ax.set_xlabel('Time Index')\n    ax.set_ylabel('Value')\n    ax.legend(fontsize=8)\n\n# Hide empty subplot\naxes[-1].set_visible(False)\n\nplt.suptitle('Different Types of Data Drift', fontsize=14, fontweight='bold')\nplt.tight_layout()\nplt.savefig('drift_types.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%% [markdown]\n# ## ส่วนที่ 3: Sliding Window Drift Detector\n# \n# Implement sliding window สำหรับ real-time drift detection\n\n#%%\nclass SlidingWindowDriftDetector:\n    \"\"\"\n    Drift detector ที่ใช้ sliding window approach\n    \n    เหมาะสำหรับ:\n    - Real-time monitoring\n    - Streaming data\n    - Memory-efficient processing\n    \"\"\"\n    \n    def __init__(self, reference_window_size=200, test_window_size=100, \n                 ks_threshold=0.05, psi_threshold=0.1):\n        \"\"\"\n        Parameters:\n        -----------\n        reference_window_size : int - ขนาดของ reference window\n        test_window_size : int - ขนาดของ test window\n        ks_threshold : float - threshold สำหรับ KS test p-value\n        psi_threshold : float - threshold สำหรับ PSI\n        \"\"\"\n        self.reference_window_size = reference_window_size\n        self.test_window_size = test_window_size\n        self.ks_threshold = ks_threshold\n        self.psi_threshold = psi_threshold\n        \n        # ใช้ deque สำหรับ efficient sliding window\n        self.reference_buffer = deque(maxlen=reference_window_size)\n        self.test_buffer = deque(maxlen=test_window_size)\n        \n        self.history = []\n        self.drift_points = []\n        self.is_initialized = False\n    \n    def calculate_psi(self, reference, test, bins=10):\n        \"\"\"คำนวณ PSI\"\"\"\n        breakpoints = np.percentile(reference, np.linspace(0, 100, bins + 1))\n        breakpoints = np.unique(breakpoints)\n        \n        ref_counts, _ = np.histogram(reference, bins=breakpoints)\n        test_counts, _ = np.histogram(test, bins=breakpoints)\n        \n        eps = 1e-6\n        ref_props = ref_counts / len(reference) + eps\n        test_props = test_counts / len(test) + eps\n        \n        psi = np.sum((test_props - ref_props) * np.log(test_props / ref_props))\n        return psi\n    \n    def update(self, value, timestamp=None):\n        \"\"\"\n        Update detector ด้วยค่าใหม่\n        \n        Returns:\n        --------\n        dict : detection result\n        \"\"\"\n        # เพิ่มค่าใน buffers\n        if not self.is_initialized:\n            self.reference_buffer.append(value)\n            if len(self.reference_buffer) >= self.reference_window_size:\n                self.is_initialized = True\n            return {'drift_detected': False, 'status': 'initializing'}\n        \n        self.test_buffer.append(value)\n        \n        # รอจนมีข้อมูลพอใน test buffer\n        if len(self.test_buffer) < self.test_window_size:\n            return {'drift_detected': False, 'status': 'collecting'}\n        \n        # ทำ drift detection\n        ref_array = np.array(self.reference_buffer)\n        test_array = np.array(self.test_buffer)\n        \n        # KS Test\n        ks_stat, ks_pval = stats.ks_2samp(ref_array, test_array)\n        \n        # PSI\n        psi = self.calculate_psi(ref_array, test_array)\n        \n        # Detection logic\n        ks_drift = ks_pval < self.ks_threshold\n        psi_drift = psi > self.psi_threshold\n        drift_detected = ks_drift or psi_drift\n        \n        result = {\n            'timestamp': timestamp,\n            'drift_detected': drift_detected,\n            'ks_statistic': ks_stat,\n            'ks_pvalue': ks_pval,\n            'psi': psi,\n            'ref_mean': np.mean(ref_array),\n            'test_mean': np.mean(test_array),\n            'status': 'DRIFT' if drift_detected else 'normal'\n        }\n        \n        self.history.append(result)\n        \n        if drift_detected:\n            self.drift_points.append(len(self.history) - 1)\n        \n        return result\n    \n    def reset_reference(self):\n        \"\"\"Reset reference window ด้วย test window ปัจจุบัน\"\"\"\n        self.reference_buffer.clear()\n        for val in self.test_buffer:\n            self.reference_buffer.append(val)\n        self.test_buffer.clear()\n        print(\"📋 Reference window reset with current data\")\n    \n    def get_history_df(self):\n        \"\"\"แปลง history เป็น DataFrame\"\"\"\n        return pd.DataFrame(self.history)\n\n#%%\n# ทดสอบ SlidingWindowDriftDetector กับ sudden drift\nprint(\"=\" * 60)\nprint(\"🔍 Testing Sliding Window Detector with SUDDEN DRIFT\")\nprint(\"=\" * 60)\n\ndetector = SlidingWindowDriftDetector(\n    reference_window_size=200,\n    test_window_size=100,\n    ks_threshold=0.05,\n    psi_threshold=0.1\n)\n\nsudden_stream = streams['sudden']\n\nfor idx, row in sudden_stream.iterrows():\n    result = detector.update(row['value'], timestamp=row['timestamp'])\n    \n    # Print only drift events\n    if result.get('drift_detected', False):\n        print(f\"⚠️ DRIFT at index {idx}: KS p-value={result['ks_pvalue']:.4f}, PSI={result['psi']:.4f}\")\n\nprint(f\"\\n📊 Total drift points detected: {len(detector.drift_points)}\")\n\n#%%\n# Visualize detection results\nhistory_df = detector.get_history_df()\n\nfig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)\n\n# Plot 1: Original data with drift points\nax1 = axes[0]\nax1.plot(sudden_stream['index'], sudden_stream['value'], alpha=0.5, label='Data')\nax1.plot(sudden_stream['index'], sudden_stream['value'].rolling(50).mean(), \n         'b-', linewidth=2, label='Rolling Mean')\n\n# Mark drift points\nfor dp in detector.drift_points:\n    ax1.axvline(dp + 200 + 100, color='red', alpha=0.5, linestyle='--')  # Offset for initialization\n\nax1.axvline(500, color='green', linestyle='--', linewidth=2, label='Actual Drift Point')\nax1.set_ylabel('Value')\nax1.set_title('Data Stream with Detected Drift Points')\nax1.legend()\n\n# Plot 2: KS p-value over time\nax2 = axes[1]\nax2.plot(history_df.index + 200 + 100, history_df['ks_pvalue'], 'b-', linewidth=1)\nax2.axhline(0.05, color='red', linestyle='--', label='Threshold (0.05)')\nax2.set_ylabel('KS p-value')\nax2.set_title('KS Test p-value Over Time')\nax2.legend()\nax2.set_yscale('log')\n\n# Plot 3: PSI over time\nax3 = axes[2]\nax3.plot(history_df.index + 200 + 100, history_df['psi'], 'g-', linewidth=1)\nax3.axhline(0.1, color='red', linestyle='--', label='Threshold (0.1)')\nax3.set_ylabel('PSI')\nax3.set_xlabel('Time Index')\nax3.set_title('PSI Over Time')\nax3.legend()\n\nplt.tight_layout()\nplt.savefig('sliding_window_detection.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%% [markdown]\n# ## ส่วนที่ 4: Detecting Gradual Drift\n# \n# Gradual drift ยากกว่า sudden drift เพราะเปลี่ยนช้าๆ\n\n#%%\nprint(\"=\" * 60)\nprint(\"🔍 Testing Sliding Window Detector with GRADUAL DRIFT\")\nprint(\"=\" * 60)\n\ndetector_gradual = SlidingWindowDriftDetector(\n    reference_window_size=200,\n    test_window_size=100\n)\n\ngradual_stream = streams['gradual']\n\nfor idx, row in gradual_stream.iterrows():\n    result = detector_gradual.update(row['value'], timestamp=row['timestamp'])\n\nhistory_gradual = detector_gradual.get_history_df()\n\nprint(f\"\\n📊 Total drift points detected: {len(detector_gradual.drift_points)}\")\nprint(f\"   First detection at index: {detector_gradual.drift_points[0] + 300 if detector_gradual.drift_points else 'N/A'}\")\n\n#%%\n# Compare detection of sudden vs gradual drift\nfig, axes = plt.subplots(2, 2, figsize=(14, 10))\n\n# Sudden drift\nax1 = axes[0, 0]\nax1.plot(sudden_stream['index'], sudden_stream['value'], alpha=0.3)\nax1.plot(sudden_stream['index'], sudden_stream['value'].rolling(50).mean(), 'b-', linewidth=2)\nfor dp in detector.drift_points:\n    ax1.axvline(dp + 300, color='red', alpha=0.3)\nax1.axvline(500, color='green', linestyle='--', linewidth=2, label='Actual Drift')\nax1.set_title('Sudden Drift Detection')\nax1.legend()\n\nax2 = axes[0, 1]\nhistory_df = detector.get_history_df()\nax2.plot(history_df.index + 300, history_df['psi'], 'g-')\nax2.axhline(0.1, color='red', linestyle='--')\nax2.set_title('Sudden Drift: PSI')\n\n# Gradual drift\nax3 = axes[1, 0]\nax3.plot(gradual_stream['index'], gradual_stream['value'], alpha=0.3)\nax3.plot(gradual_stream['index'], gradual_stream['value'].rolling(50).mean(), 'b-', linewidth=2)\nfor dp in detector_gradual.drift_points:\n    ax3.axvline(dp + 300, color='red', alpha=0.3)\nax3.axvspan(200, 800, alpha=0.2, color='green', label='Drift Period')\nax3.set_title('Gradual Drift Detection')\nax3.legend()\n\nax4 = axes[1, 1]\nax4.plot(history_gradual.index + 300, history_gradual['psi'], 'g-')\nax4.axhline(0.1, color='red', linestyle='--')\nax4.set_title('Gradual Drift: PSI')\n\nplt.tight_layout()\nplt.savefig('sudden_vs_gradual_detection.png', dpi=150, bbox_inches='tight')\nplt.show()\n\nprint(\"\\n💡 Observation:\")\nprint(\"   - Sudden drift: ตรวจจับได้เร็วและชัดเจน\")\nprint(\"   - Gradual drift: ใช้เวลานานกว่าจะ detect ได้\")\n\n#%% [markdown]\n# ## ส่วนที่ 5: Adaptive Reference Window\n# \n# ปรับ reference window เพื่อ handle gradual drift\n\n#%%\nclass AdaptiveDriftDetector:\n    \"\"\"\n    Drift detector ที่ปรับ reference window อัตโนมัติ\n    \n    Features:\n    - ปรับ reference เมื่อตรวจจับ drift\n    - ป้องกัน false alarms จาก temporary spikes\n    - Track both short-term และ long-term drift\n    \"\"\"\n    \n    def __init__(self, reference_window_size=200, test_window_size=50,\n                 confirmation_window=3, psi_threshold=0.1):\n        \"\"\"\n        Parameters:\n        -----------\n        confirmation_window : int - จำนวนครั้งติดต่อกันที่ต้อง detect ก่อนยืนยัน drift\n        \"\"\"\n        self.reference_window_size = reference_window_size\n        self.test_window_size = test_window_size\n        self.confirmation_window = confirmation_window\n        self.psi_threshold = psi_threshold\n        \n        self.reference_buffer = deque(maxlen=reference_window_size)\n        self.test_buffer = deque(maxlen=test_window_size)\n        \n        self.consecutive_drift_count = 0\n        self.confirmed_drifts = []\n        self.history = []\n        self.adaptation_count = 0\n    \n    def calculate_psi(self, reference, test, bins=10):\n        \"\"\"คำนวณ PSI\"\"\"\n        breakpoints = np.percentile(reference, np.linspace(0, 100, bins + 1))\n        breakpoints = np.unique(breakpoints)\n        \n        ref_counts, _ = np.histogram(reference, bins=breakpoints)\n        test_counts, _ = np.histogram(test, bins=breakpoints)\n        \n        eps = 1e-6\n        ref_props = ref_counts / len(reference) + eps\n        test_props = test_counts / len(test) + eps\n        \n        psi = np.sum((test_props - ref_props) * np.log(test_props / ref_props))\n        return psi\n    \n    def adapt_reference(self):\n        \"\"\"ปรับ reference window\"\"\"\n        # ผสม old reference กับ new data\n        old_weight = 0.5\n        old_ref = list(self.reference_buffer)\n        new_data = list(self.test_buffer)\n        \n        # สร้าง new reference\n        self.reference_buffer.clear()\n        \n        # เพิ่มบางส่วนจาก old reference\n        n_old = int(len(old_ref) * old_weight)\n        for val in old_ref[-n_old:]:\n            self.reference_buffer.append(val)\n        \n        # เพิ่ม new data\n        for val in new_data:\n            self.reference_buffer.append(val)\n        \n        self.adaptation_count += 1\n        self.consecutive_drift_count = 0\n    \n    def update(self, value, timestamp=None):\n        \"\"\"Update detector\"\"\"\n        # Initialize\n        if len(self.reference_buffer) < self.reference_window_size:\n            self.reference_buffer.append(value)\n            return {'status': 'initializing', 'drift_detected': False}\n        \n        self.test_buffer.append(value)\n        \n        if len(self.test_buffer) < self.test_window_size:\n            return {'status': 'collecting', 'drift_detected': False}\n        \n        # Drift detection\n        ref_array = np.array(self.reference_buffer)\n        test_array = np.array(self.test_buffer)\n        \n        psi = self.calculate_psi(ref_array, test_array)\n        potential_drift = psi > self.psi_threshold\n        \n        if potential_drift:\n            self.consecutive_drift_count += 1\n        else:\n            self.consecutive_drift_count = 0\n        \n        # Confirmed drift (multiple consecutive detections)\n        confirmed = self.consecutive_drift_count >= self.confirmation_window\n        \n        result = {\n            'timestamp': timestamp,\n            'psi': psi,\n            'potential_drift': potential_drift,\n            'confirmed_drift': confirmed,\n            'consecutive_count': self.consecutive_drift_count,\n            'adaptation_count': self.adaptation_count,\n            'ref_mean': np.mean(ref_array),\n            'test_mean': np.mean(test_array)\n        }\n        \n        self.history.append(result)\n        \n        # Adapt if confirmed\n        if confirmed:\n            self.confirmed_drifts.append(len(self.history) - 1)\n            self.adapt_reference()\n            result['adapted'] = True\n        \n        return result\n    \n    def get_history_df(self):\n        return pd.DataFrame(self.history)\n\n#%%\n# ทดสอบ Adaptive Detector\nprint(\"=\" * 60)\nprint(\"🔍 Testing ADAPTIVE Drift Detector with GRADUAL DRIFT\")\nprint(\"=\" * 60)\n\nadaptive_detector = AdaptiveDriftDetector(\n    reference_window_size=200,\n    test_window_size=50,\n    confirmation_window=3,\n    psi_threshold=0.1\n)\n\nfor idx, row in gradual_stream.iterrows():\n    result = adaptive_detector.update(row['value'], timestamp=row['timestamp'])\n    \n    if result.get('confirmed_drift', False):\n        print(f\"✅ CONFIRMED DRIFT at index {idx}: PSI={result['psi']:.4f}, Adapted!\")\n\nprint(f\"\\n📊 Total confirmed drifts: {len(adaptive_detector.confirmed_drifts)}\")\nprint(f\"📊 Total adaptations: {adaptive_detector.adaptation_count}\")\n\n#%%\n# Visualize adaptive detection\nadaptive_history = adaptive_detector.get_history_df()\n\nfig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)\n\n# Plot 1: Data with adaptations\nax1 = axes[0]\nax1.plot(gradual_stream['index'], gradual_stream['value'], alpha=0.3)\nax1.plot(gradual_stream['index'], gradual_stream['value'].rolling(50).mean(), 'b-', linewidth=2)\n\noffset = 200 + 50  # initialization + test window\nfor drift_idx in adaptive_detector.confirmed_drifts:\n    ax1.axvline(drift_idx + offset, color='red', linestyle='--', alpha=0.7, linewidth=2)\n\nax1.set_title('Gradual Drift with Adaptive Detection')\nax1.set_ylabel('Value')\n\n# Plot 2: PSI with threshold\nax2 = axes[1]\nax2.plot(adaptive_history.index + offset, adaptive_history['psi'], 'g-')\nax2.axhline(0.1, color='red', linestyle='--', label='Threshold')\nax2.set_ylabel('PSI')\nax2.legend()\n\n# Plot 3: Reference vs Test mean\nax3 = axes[2]\nax3.plot(adaptive_history.index + offset, adaptive_history['ref_mean'], 'b-', label='Reference Mean')\nax3.plot(adaptive_history.index + offset, adaptive_history['test_mean'], 'r-', label='Test Mean')\nax3.set_xlabel('Time Index')\nax3.set_ylabel('Mean')\nax3.legend()\n\nplt.tight_layout()\nplt.savefig('adaptive_detection.png', dpi=150, bbox_inches='tight')\nplt.show()\n\nprint(\"\\n💡 Adaptive detector ปรับ reference window เมื่อ detect drift\")\nprint(\"   ทำให้สามารถติดตาม gradual drift ได้ต่อเนื่อง\")\n\n#%% [markdown]\n# ## ส่วนที่ 6: Page-Hinkley Test for Change Detection\n# \n# Algorithm สำหรับ detect mean shift ใน streaming data\n\n#%%\nclass PageHinkleyDetector:\n    \"\"\"\n    Page-Hinkley test สำหรับ detect mean shift\n    \n    เหมาะสำหรับ:\n    - Streaming data\n    - Detect upward หรือ downward shifts\n    - Low memory footprint\n    \"\"\"\n    \n    def __init__(self, delta=0.005, lambda_=50, alpha=0.9999):\n        \"\"\"\n        Parameters:\n        -----------\n        delta : float - magnitude ที่ยอมรับได้ของ change\n        lambda_ : float - detection threshold\n        alpha : float - forgetting factor สำหรับ mean estimation\n        \"\"\"\n        self.delta = delta\n        self.lambda_ = lambda_\n        self.alpha = alpha\n        \n        self.mean = 0\n        self.sum = 0\n        self.min_sum = float('inf')\n        self.max_sum = float('-inf')\n        \n        self.n_samples = 0\n        self.history = []\n        self.drift_points = []\n    \n    def update(self, value, timestamp=None):\n        \"\"\"\n        Update detector ด้วยค่าใหม่\n        \"\"\"\n        self.n_samples += 1\n        \n        # Update mean\n        if self.n_samples == 1:\n            self.mean = value\n        else:\n            self.mean = self.alpha * self.mean + (1 - self.alpha) * value\n        \n        # Update cumulative sum\n        self.sum += value - self.mean - self.delta\n        \n        # Update min/max\n        self.min_sum = min(self.min_sum, self.sum)\n        self.max_sum = max(self.max_sum, self.sum)\n        \n        # Calculate test statistics\n        ph_positive = self.sum - self.min_sum  # Detect upward shift\n        ph_negative = self.max_sum - self.sum  # Detect downward shift\n        \n        # Detection\n        drift_up = ph_positive > self.lambda_\n        drift_down = ph_negative > self.lambda_\n        drift_detected = drift_up or drift_down\n        \n        result = {\n            'timestamp': timestamp,\n            'value': value,\n            'mean': self.mean,\n            'sum': self.sum,\n            'ph_positive': ph_positive,\n            'ph_negative': ph_negative,\n            'drift_detected': drift_detected,\n            'drift_direction': 'up' if drift_up else ('down' if drift_down else None)\n        }\n        \n        self.history.append(result)\n        \n        if drift_detected:\n            self.drift_points.append(self.n_samples - 1)\n            self.reset()\n        \n        return result\n    \n    def reset(self):\n        \"\"\"Reset statistics หลัง detect drift\"\"\"\n        self.sum = 0\n        self.min_sum = float('inf')\n        self.max_sum = float('-inf')\n    \n    def get_history_df(self):\n        return pd.DataFrame(self.history)\n\n#%%\n# ทดสอบ Page-Hinkley\nprint(\"=\" * 60)\nprint(\"🔍 Testing PAGE-HINKLEY Detector\")\nprint(\"=\" * 60)\n\nph_detector = PageHinkleyDetector(delta=0.01, lambda_=30)\n\n# ทดสอบกับ sudden drift\nfor idx, row in sudden_stream.iterrows():\n    result = ph_detector.update(row['value'], timestamp=row['timestamp'])\n    \n    if result['drift_detected']:\n        print(f\"⚠️ DRIFT at index {idx}: Direction = {result['drift_direction']}\")\n\nprint(f\"\\n📊 Total drifts detected: {len(ph_detector.drift_points)}\")\n\n#%%\n# Visualize Page-Hinkley results\nph_history = ph_detector.get_history_df()\n\nfig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)\n\n# Plot 1: Data\nax1 = axes[0]\nax1.plot(sudden_stream['index'], sudden_stream['value'], alpha=0.3)\nax1.plot(ph_history.index, ph_history['mean'], 'r-', linewidth=2, label='Estimated Mean')\nfor dp in ph_detector.drift_points:\n    ax1.axvline(dp, color='red', linestyle='--', alpha=0.7)\nax1.set_title('Page-Hinkley Detection')\nax1.set_ylabel('Value')\nax1.legend()\n\n# Plot 2: PH Statistics\nax2 = axes[1]\nax2.plot(ph_history.index, ph_history['ph_positive'], 'g-', label='PH+ (upward)')\nax2.plot(ph_history.index, ph_history['ph_negative'], 'b-', label='PH- (downward)')\nax2.axhline(30, color='red', linestyle='--', label='Threshold')\nax2.set_ylabel('PH Statistics')\nax2.legend()\n\n# Plot 3: Cumulative Sum\nax3 = axes[2]\nax3.plot(ph_history.index, ph_history['sum'], 'purple')\nax3.set_xlabel('Time Index')\nax3.set_ylabel('Cumulative Sum')\n\nplt.tight_layout()\nplt.savefig('page_hinkley_detection.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%% [markdown]\n# ## สรุป LAB 4\n# \n# ### สิ่งที่เรียนรู้:\n# 1. **Data Stream Simulation**: สร้าง streams ที่มี drift patterns ต่างๆ\n# 2. **Sliding Window**: วิธีมาตรฐานสำหรับ streaming drift detection\n# 3. **Adaptive Detection**: ปรับ reference window เมื่อ detect drift\n# 4. **Page-Hinkley**: Algorithm สำหรับ mean shift detection\n# \n# ### Comparison:\n# | Method | Pros | Cons | Best For |\n# |--------|------|------|----------|\n# | Sliding Window | Simple, intuitive | Fixed reference | Sudden drift |\n# | Adaptive | Handles gradual drift | More complex | Production |\n# | Page-Hinkley | Low memory, fast | Mean shift only | Real-time |\n\n#%%\nprint(\"=\" * 60)\nprint(\"✅ LAB 4 COMPLETED!\")\nprint(\"=\" * 60)\nprint(\"\"\"\n📚 Key Takeaways:\n1. Different drift types require different detection strategies\n2. Sliding window is fundamental for streaming detection\n3. Adaptive reference helps with gradual drift\n4. Page-Hinkley is efficient for mean shift detection\n\n🔜 Next: LAB 5 - Custom Metrics & Drift Thresholds\n\"\"\")\n```\n\n---\n\n## LAB 5: Custom Metrics & Drift Thresholds\n\n```python\n#%% [markdown]\n# # LAB 5: Custom Metrics & Drift Thresholds\n# ## การสร้าง Custom Drift Metrics และปรับ Thresholds\n# \n# ### วัตถุประสงค์การเรียนรู้:\n# 1. สร้าง custom drift metrics ที่เหมาะกับ domain\n# 2. ปรับ threshold ตาม business requirements\n# 3. Handle false positives/negatives ใน drift detection\n#\n# ### ทฤษฎี:\n# Default thresholds อาจไม่เหมาะกับทุก use case:\n# - บาง domain ต้องการ sensitivity สูง\n# - บาง domain ยอมรับ drift ได้ระดับหนึ่ง\n# - Cost of false positive vs false negative ต่างกัน\n\n#%% [markdown]\n# ## ส่วนที่ 1: เตรียม Environment\n\n#%%\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom scipy import stats\nfrom sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score\nimport warnings\nwarnings.filterwarnings('ignore')\n\nnp.random.seed(42)\nplt.rcParams['figure.figsize'] = (12, 6)\n\nprint(\"✅ Libraries imported successfully!\")\n\n#%% [markdown]\n# ## ส่วนที่ 2: สร้าง Dataset สำหรับ Threshold Tuning\n# \n# สร้าง labeled dataset ที่รู้ว่ามี drift หรือไม่\n\n#%%\ndef create_labeled_drift_dataset(n_scenarios=100):\n    \"\"\"\n    สร้าง dataset พร้อม ground truth labels\n    \n    Returns:\n    --------\n    list of dicts: แต่ละ scenario มี reference, current, และ label\n    \"\"\"\n    scenarios = []\n    \n    for i in range(n_scenarios):\n        np.random.seed(i)\n        n_samples = 500\n        \n        # Reference data\n        ref_mean = 50\n        ref_std = 10\n        reference = np.random.normal(ref_mean, ref_std, n_samples)\n        \n        # กำหนดว่ามี drift หรือไม่\n        has_drift = i < n_scenarios // 2  # 50% มี drift\n        \n        if has_drift:\n            # สร้าง drift ในระดับต่างๆ\n            drift_level = (i % 5 + 1) * 0.5  # 0.5, 1.0, 1.5, 2.0, 2.5 std\n            current_mean = ref_mean + drift_level * ref_std\n            drift_magnitude = drift_level\n        else:\n            # ไม่มี drift แต่มี noise เล็กน้อย\n            current_mean = ref_mean + np.random.uniform(-0.1, 0.1) * ref_std\n            drift_magnitude = 0\n        \n        current = np.random.normal(current_mean, ref_std, n_samples)\n        \n        scenarios.append({\n            'id': i,\n            'reference': reference,\n            'current': current,\n            'has_drift': has_drift,\n            'drift_magnitude': drift_magnitude,\n            'ref_mean': ref_mean,\n            'current_mean': current_mean\n        })\n    \n    return scenarios\n\n# สร้าง dataset\nscenarios = create_labeled_drift_dataset(100)\n\nprint(f\"📊 Created {len(scenarios)} scenarios\")\nprint(f\"   With drift: {sum(1 for s in scenarios if s['has_drift'])}\")\nprint(f\"   Without drift: {sum(1 for s in scenarios if not s['has_drift'])}\")\n\n#%%\n# Visualize drift magnitude distribution\ndrift_mags = [s['drift_magnitude'] for s in scenarios if s['has_drift']]\nplt.figure(figsize=(10, 4))\nplt.hist(drift_mags, bins=10, edgecolor='black', alpha=0.7)\nplt.xlabel('Drift Magnitude (in std)')\nplt.ylabel('Count')\nplt.title('Distribution of Drift Magnitudes in Scenarios with Drift')\nplt.show()\n\n#%% [markdown]\n# ## ส่วนที่ 3: สร้าง Custom Drift Metrics\n\n#%%\nclass CustomDriftMetrics:\n    \"\"\"\n    Class สำหรับคำนวณ custom drift metrics\n    \"\"\"\n    \n    @staticmethod\n    def psi(reference, current, bins=10):\n        \"\"\"Population Stability Index\"\"\"\n        breakpoints = np.percentile(reference, np.linspace(0, 100, bins + 1))\n        breakpoints = np.unique(breakpoints)\n        \n        ref_counts, _ = np.histogram(reference, bins=breakpoints)\n        cur_counts, _ = np.histogram(current, bins=breakpoints)\n        \n        eps = 1e-6\n        ref_props = ref_counts / len(reference) + eps\n        cur_props = cur_counts / len(current) + eps\n        \n        return np.sum((cur_props - ref_props) * np.log(cur_props / ref_props))\n    \n    @staticmethod\n    def normalized_wasserstein(reference, current):\n        \"\"\"Wasserstein distance normalized by reference std\"\"\"\n        distance = stats.wasserstein_distance(reference, current)\n        return distance / np.std(reference)\n    \n    @staticmethod\n    def mean_shift_ratio(reference, current):\n        \"\"\"Mean shift as ratio of reference std\"\"\"\n        mean_diff = abs(np.mean(current) - np.mean(reference))\n        return mean_diff / np.std(reference)\n    \n    @staticmethod\n    def std_ratio(reference, current):\n        \"\"\"Ratio of standard deviations\"\"\"\n        return np.std(current) / np.std(reference)\n    \n    @staticmethod\n    def percentile_shift(reference, current, percentiles=[25, 50, 75]):\n        \"\"\"Average shift in percentiles\"\"\"\n        shifts = []\n        ref_std = np.std(reference)\n        \n        for p in percentiles:\n            ref_p = np.percentile(reference, p)\n            cur_p = np.percentile(current, p)\n            shifts.append(abs(cur_p - ref_p) / ref_std)\n        \n        return np.mean(shifts)\n    \n    @staticmethod\n    def jensen_shannon_divergence(reference, current, bins=10):\n        \"\"\"Jensen-Shannon divergence\"\"\"\n        # สร้าง histograms\n        all_data = np.concatenate([reference, current])\n        bins = np.histogram_bin_edges(all_data, bins=bins)\n        \n        ref_hist, _ = np.histogram(reference, bins=bins, density=True)\n        cur_hist, _ = np.histogram(current, bins=bins, density=True)\n        \n        # Normalize\n        ref_hist = ref_hist / (ref_hist.sum() + 1e-10)\n        cur_hist = cur_hist / (cur_hist.sum() + 1e-10)\n        \n        # Average distribution\n        m = 0.5 * (ref_hist + cur_hist)\n        \n        # KL divergences\n        kl_pm = np.sum(ref_hist * np.log((ref_hist + 1e-10) / (m + 1e-10)))\n        kl_qm = np.sum(cur_hist * np.log((cur_hist + 1e-10) / (m + 1e-10)))\n        \n        return 0.5 * (kl_pm + kl_qm)\n    \n    @staticmethod\n    def combined_score(reference, current, weights=None):\n        \"\"\"\n        Combined drift score จากหลาย metrics\n        \n        Default weights: PSI=0.3, Wasserstein=0.3, Mean Shift=0.2, Percentile=0.2\n        \"\"\"\n        if weights is None:\n            weights = {\n                'psi': 0.3,\n                'wasserstein': 0.3,\n                'mean_shift': 0.2,\n                'percentile': 0.2\n            }\n        \n        metrics = CustomDriftMetrics\n        \n        # Normalize each metric to 0-1 range\n        psi = min(metrics.psi(reference, current), 1.0)  # Cap at 1\n        wasserstein = min(metrics.normalized_wasserstein(reference, current) / 3, 1.0)  # 3 std = max\n        mean_shift = min(metrics.mean_shift_ratio(reference, current) / 3, 1.0)\n        percentile = min(metrics.percentile_shift(reference, current) / 3, 1.0)\n        \n        score = (\n            weights['psi'] * psi +\n            weights['wasserstein'] * wasserstein +\n            weights['mean_shift'] * mean_shift +\n            weights['percentile'] * percentile\n        )\n        \n        return score\n\n#%%\n# ทดสอบ custom metrics\nprint(\"=\" * 60)\nprint(\"📊 Testing Custom Drift Metrics\")\nprint(\"=\" * 60)\n\n# เลือก scenarios ที่มี drift levels ต่างๆ\ntest_scenarios = [s for s in scenarios if s['drift_magnitude'] in [0, 0.5, 1.0, 2.0]][:8]\n\nmetrics_results = []\nfor s in test_scenarios:\n    result = {\n        'id': s['id'],\n        'has_drift': s['has_drift'],\n        'magnitude': s['drift_magnitude'],\n        'psi': CustomDriftMetrics.psi(s['reference'], s['current']),\n        'wasserstein': CustomDriftMetrics.normalized_wasserstein(s['reference'], s['current']),\n        'mean_shift': CustomDriftMetrics.mean_shift_ratio(s['reference'], s['current']),\n        'percentile': CustomDriftMetrics.percentile_shift(s['reference'], s['current']),\n        'js_div': CustomDriftMetrics.jensen_shannon_divergence(s['reference'], s['current']),\n        'combined': CustomDriftMetrics.combined_score(s['reference'], s['current'])\n    }\n    metrics_results.append(result)\n\nmetrics_df = pd.DataFrame(metrics_results)\nprint(metrics_df.to_string(index=False))\n\n#%% [markdown]\n# ## ส่วนที่ 4: Threshold Optimization\n# \n# หา optimal threshold โดยใช้ labeled data\n\n#%%\ndef calculate_metrics_for_threshold(scenarios, metric_func, threshold):\n    \"\"\"\n    คำนวณ precision, recall, f1 สำหรับ threshold ที่กำหนด\n    \"\"\"\n    y_true = []\n    y_pred = []\n    \n    for s in scenarios:\n        y_true.append(1 if s['has_drift'] else 0)\n        \n        score = metric_func(s['reference'], s['current'])\n        y_pred.append(1 if score > threshold else 0)\n    \n    # Handle edge cases\n    if sum(y_pred) == 0:\n        return {'precision': 0, 'recall': 0, 'f1': 0}\n    \n    precision = precision_score(y_true, y_pred, zero_division=0)\n    recall = recall_score(y_true, y_pred, zero_division=0)\n    f1 = f1_score(y_true, y_pred, zero_division=0)\n    \n    return {'precision': precision, 'recall': recall, 'f1': f1}\n\ndef find_optimal_threshold(scenarios, metric_func, thresholds, optimize_for='f1'):\n    \"\"\"\n    หา optimal threshold\n    \"\"\"\n    results = []\n    \n    for t in thresholds:\n        metrics = calculate_metrics_for_threshold(scenarios, metric_func, t)\n        metrics['threshold'] = t\n        results.append(metrics)\n    \n    results_df = pd.DataFrame(results)\n    \n    # หา optimal\n    optimal_idx = results_df[optimize_for].idxmax()\n    optimal = results_df.iloc[optimal_idx]\n    \n    return results_df, optimal\n\n#%%\n# หา optimal threshold สำหรับ PSI\nthresholds = np.linspace(0.01, 0.5, 50)\n\npsi_results, psi_optimal = find_optimal_threshold(\n    scenarios, \n    CustomDriftMetrics.psi, \n    thresholds\n)\n\nprint(\"=\" * 60)\nprint(\"📊 PSI Threshold Optimization\")\nprint(\"=\" * 60)\nprint(f\"Optimal threshold: {psi_optimal['threshold']:.3f}\")\nprint(f\"Precision: {psi_optimal['precision']:.3f}\")\nprint(f\"Recall: {psi_optimal['recall']:.3f}\")\nprint(f\"F1 Score: {psi_optimal['f1']:.3f}\")\n\n#%%\n# Visualize threshold optimization\nfig, axes = plt.subplots(1, 3, figsize=(15, 4))\n\nmetrics_to_plot = ['precision', 'recall', 'f1']\ncolors = ['blue', 'green', 'red']\n\nfor ax, metric, color in zip(axes, metrics_to_plot, colors):\n    ax.plot(psi_results['threshold'], psi_results[metric], f'{color}-', linewidth=2)\n    ax.axvline(psi_optimal['threshold'], color='black', linestyle='--', \n               label=f'Optimal ({psi_optimal[\"threshold\"]:.3f})')\n    ax.set_xlabel('PSI Threshold')\n    ax.set_ylabel(metric.capitalize())\n    ax.set_title(f'{metric.capitalize()} vs Threshold')\n    ax.legend()\n    ax.grid(True, alpha=0.3)\n\nplt.suptitle('PSI Threshold Optimization', fontsize=14, fontweight='bold')\nplt.tight_layout()\nplt.savefig('threshold_optimization_psi.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%%\n# เปรียบเทียบ optimal thresholds ของแต่ละ metric\nprint(\"=\" * 60)\nprint(\"📊 Comparing Optimal Thresholds for Different Metrics\")\nprint(\"=\" * 60)\n\nmetrics_funcs = {\n    'PSI': CustomDriftMetrics.psi,\n    'Normalized Wasserstein': CustomDriftMetrics.normalized_wasserstein,\n    'Mean Shift Ratio': CustomDriftMetrics.mean_shift_ratio,\n    'Combined Score': CustomDriftMetrics.combined_score\n}\n\nthreshold_ranges = {\n    'PSI': np.linspace(0.01, 0.5, 50),\n    'Normalized Wasserstein': np.linspace(0.01, 2.0, 50),\n    'Mean Shift Ratio': np.linspace(0.01, 2.0, 50),\n    'Combined Score': np.linspace(0.01, 0.5, 50)\n}\n\noptimal_thresholds = {}\nfor name, func in metrics_funcs.items():\n    results, optimal = find_optimal_threshold(scenarios, func, threshold_ranges[name])\n    optimal_thresholds[name] = optimal\n    print(f\"\\n{name}:\")\n    print(f\"  Optimal threshold: {optimal['threshold']:.3f}\")\n    print(f\"  F1 Score: {optimal['f1']:.3f}\")\n\n#%% [markdown]\n# ## ส่วนที่ 5: Business-driven Threshold Setting\n# \n# ปรับ threshold ตาม business requirements\n\n#%%\nclass BusinessDriftThreshold:\n    \"\"\"\n    Class สำหรับกำหนด threshold ตาม business context\n    \"\"\"\n    \n    def __init__(self, false_positive_cost=1, false_negative_cost=10):\n        \"\"\"\n        Parameters:\n        -----------\n        false_positive_cost : float - cost ของ false alarm\n        false_negative_cost : float - cost ของ missing drift\n        \"\"\"\n        self.fp_cost = false_positive_cost\n        self.fn_cost = false_negative_cost\n    \n    def calculate_total_cost(self, y_true, y_pred):\n        \"\"\"คำนวณ total cost\"\"\"\n        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()\n        return fp * self.fp_cost + fn * self.fn_cost\n    \n    def find_cost_optimal_threshold(self, scenarios, metric_func, thresholds):\n        \"\"\"หา threshold ที่ minimize total cost\"\"\"\n        costs = []\n        \n        for t in thresholds:\n            y_true = [1 if s['has_drift'] else 0 for s in scenarios]\n            y_pred = [1 if metric_func(s['reference'], s['current']) > t else 0 for s in scenarios]\n            \n            cost = self.calculate_total_cost(y_true, y_pred)\n            costs.append({'threshold': t, 'cost': cost})\n        \n        cost_df = pd.DataFrame(costs)\n        optimal_idx = cost_df['cost'].idxmin()\n        return cost_df, cost_df.iloc[optimal_idx]\n\n#%%\n# เปรียบเทียบ threshold สำหรับ scenarios ต่างๆ\nprint(\"=\" * 60)\nprint(\"📊 Business-driven Threshold Optimization\")\nprint(\"=\" * 60)\n\n# Scenario 1: High cost of missing drift (e.g., fraud detection)\nprint(\"\\n🔴 Scenario 1: High cost of missing drift (FN cost = 10x)\")\nhigh_fn_cost = BusinessDriftThreshold(false_positive_cost=1, false_negative_cost=10)\ncost_df_1, optimal_1 = high_fn_cost.find_cost_optimal_threshold(\n    scenarios, CustomDriftMetrics.psi, thresholds\n)\nprint(f\"   Optimal threshold: {optimal_1['threshold']:.3f}\")\nprint(f\"   Total cost: {optimal_1['cost']:.0f}\")\n\n# Scenario 2: High cost of false alarms (e.g., model retraining is expensive)\nprint(\"\\n🟡 Scenario 2: High cost of false alarms (FP cost = 10x)\")\nhigh_fp_cost = BusinessDriftThreshold(false_positive_cost=10, false_negative_cost=1)\ncost_df_2, optimal_2 = high_fp_cost.find_cost_optimal_threshold(\n    scenarios, CustomDriftMetrics.psi, thresholds\n)\nprint(f\"   Optimal threshold: {optimal_2['threshold']:.3f}\")\nprint(f\"   Total cost: {optimal_2['cost']:.0f}\")\n\n# Scenario 3: Balanced costs\nprint(\"\\n🟢 Scenario 3: Balanced costs (FP = FN)\")\nbalanced_cost = BusinessDriftThreshold(false_positive_cost=1, false_negative_cost=1)\ncost_df_3, optimal_3 = balanced_cost.find_cost_optimal_threshold(\n    scenarios, CustomDriftMetrics.psi, thresholds\n)\nprint(f\"   Optimal threshold: {optimal_3['threshold']:.3f}\")\nprint(f\"   Total cost: {optimal_3['cost']:.0f}\")\n\n#%%\n# Visualize cost-based optimization\nfig, axes = plt.subplots(1, 3, figsize=(15, 4))\n\nscenarios_data = [\n    (cost_df_1, optimal_1, 'High FN Cost (10x)', 'red'),\n    (cost_df_2, optimal_2, 'High FP Cost (10x)', 'orange'),\n    (cost_df_3, optimal_3, 'Balanced Cost', 'green')\n]\n\nfor ax, (cost_df, optimal, title, color) in zip(axes, scenarios_data):\n    ax.plot(cost_df['threshold'], cost_df['cost'], f'{color[0]}-', linewidth=2)\n    ax.axvline(optimal['threshold'], color='black', linestyle='--',\n               label=f'Optimal ({optimal[\"threshold\"]:.3f})')\n    ax.scatter([optimal['threshold']], [optimal['cost']], color='black', s=100, zorder=5)\n    ax.set_xlabel('PSI Threshold')\n    ax.set_ylabel('Total Cost')\n    ax.set_title(title)\n    ax.legend()\n    ax.grid(True, alpha=0.3)\n\nplt.tight_layout()\nplt.savefig('cost_based_optimization.png', dpi=150, bbox_inches='tight')\nplt.show()\n\nprint(\"\\n💡 Insight:\")\nprint(\"   - High FN cost → Lower threshold (detect more, accept false alarms)\")\nprint(\"   - High FP cost → Higher threshold (be more conservative)\")\nprint(\"   - Balanced → Somewhere in between\")\n\n#%% [markdown]\n# ## ส่วนที่ 6: Handling False Positives/Negatives\n\n#%%\nclass RobustDriftDetector:\n    \"\"\"\n    Drift detector ที่มี mechanisms สำหรับ handle FP/FN\n    \"\"\"\n    \n    def __init__(self, primary_metric='psi', primary_threshold=0.1,\n                 confirmation_window=3, use_ensemble=True):\n        \"\"\"\n        Parameters:\n        -----------\n        primary_metric : str - metric หลัก\n        primary_threshold : float - threshold หลัก\n        confirmation_window : int - ต้อง detect กี่ครั้งติดกัน\n        use_ensemble : bool - ใช้หลาย metrics ร่วมกัน\n        \"\"\"\n        self.primary_metric = primary_metric\n        self.primary_threshold = primary_threshold\n        self.confirmation_window = confirmation_window\n        self.use_ensemble = use_ensemble\n        \n        self.metrics = CustomDriftMetrics()\n        \n        # Thresholds for ensemble\n        self.thresholds = {\n            'psi': 0.1,\n            'wasserstein': 0.5,\n            'mean_shift': 0.5,\n            'percentile': 0.3\n        }\n        \n        self.consecutive_count = 0\n        self.history = []\n    \n    def _calculate_all_metrics(self, reference, current):\n        \"\"\"คำนวณทุก metrics\"\"\"\n        return {\n            'psi': self.metrics.psi(reference, current),\n            'wasserstein': self.metrics.normalized_wasserstein(reference, current),\n            'mean_shift': self.metrics.mean_shift_ratio(reference, current),\n            'percentile': self.metrics.percentile_shift(reference, current)\n        }\n    \n    def detect(self, reference, current):\n        \"\"\"\n        Detect drift with robustness mechanisms\n        \"\"\"\n        all_metrics = self._calculate_all_metrics(reference, current)\n        \n        # Single metric detection\n        primary_value = all_metrics[self.primary_metric]\n        primary_drift = primary_value > self.primary_threshold\n        \n        if self.use_ensemble:\n            # Ensemble: majority voting\n            drift_votes = sum(\n                1 for m, v in all_metrics.items() \n                if v > self.thresholds.get(m, 0.5)\n            )\n            ensemble_drift = drift_votes >= 3  # ต้อง 3/4 metrics agree\n        else:\n            ensemble_drift = primary_drift\n        \n        # Confirmation mechanism\n        if ensemble_drift:\n            self.consecutive_count += 1\n        else:\n            self.consecutive_count = 0\n        \n        confirmed_drift = self.consecutive_count >= self.confirmation_window\n        \n        result = {\n            'metrics': all_metrics,\n            'primary_drift': primary_drift,\n            'ensemble_drift': ensemble_drift,\n            'consecutive_count': self.consecutive_count,\n            'confirmed_drift': confirmed_drift\n        }\n        \n        self.history.append(result)\n        \n        return result\n    \n    def evaluate(self, scenarios):\n        \"\"\"\n        Evaluate detector performance on labeled scenarios\n        \"\"\"\n        y_true = []\n        y_pred_primary = []\n        y_pred_ensemble = []\n        y_pred_confirmed = []\n        \n        for s in scenarios:\n            y_true.append(1 if s['has_drift'] else 0)\n            \n            result = self.detect(s['reference'], s['current'])\n            y_pred_primary.append(1 if result['primary_drift'] else 0)\n            y_pred_ensemble.append(1 if result['ensemble_drift'] else 0)\n            y_pred_confirmed.append(1 if result['confirmed_drift'] else 0)\n            \n            # Reset for next scenario\n            self.consecutive_count = 0\n        \n        evaluations = {}\n        for name, y_pred in [\n            ('primary', y_pred_primary),\n            ('ensemble', y_pred_ensemble),\n            ('confirmed', y_pred_confirmed)\n        ]:\n            evaluations[name] = {\n                'precision': precision_score(y_true, y_pred, zero_division=0),\n                'recall': recall_score(y_true, y_pred, zero_division=0),\n                'f1': f1_score(y_true, y_pred, zero_division=0)\n            }\n        \n        return evaluations\n\n#%%\n# ทดสอบ RobustDriftDetector\nprint(\"=\" * 60)\nprint(\"📊 Evaluating Robust Drift Detector\")\nprint(\"=\" * 60)\n\ndetector = RobustDriftDetector(\n    primary_metric='psi',\n    primary_threshold=0.1,\n    confirmation_window=1,  # ลดเหลือ 1 สำหรับ single scenario testing\n    use_ensemble=True\n)\n\nevaluations = detector.evaluate(scenarios)\n\nfor method, metrics in evaluations.items():\n    print(f\"\\n{method.upper()} Method:\")\n    print(f\"  Precision: {metrics['precision']:.3f}\")\n    print(f\"  Recall: {metrics['recall']:.3f}\")\n    print(f\"  F1 Score: {metrics['f1']:.3f}\")\n\n#%%\n# Visualize comparison\nfig, ax = plt.subplots(figsize=(10, 6))\n\nmethods = list(evaluations.keys())\nx = np.arange(len(methods))\nwidth = 0.25\n\nmetrics_names = ['precision', 'recall', 'f1']\ncolors = ['blue', 'green', 'red']\n\nfor i, (metric, color) in enumerate(zip(metrics_names, colors)):\n    values = [evaluations[m][metric] for m in methods]\n    ax.bar(x + i * width, values, width, label=metric.capitalize(), color=color, alpha=0.7)\n\nax.set_ylabel('Score')\nax.set_xlabel('Detection Method')\nax.set_title('Comparison of Detection Methods')\nax.set_xticks(x + width)\nax.set_xticklabels([m.capitalize() for m in methods])\nax.legend()\nax.set_ylim(0, 1)\n\nfor i, (metric, color) in enumerate(zip(metrics_names, colors)):\n    values = [evaluations[m][metric] for m in methods]\n    for j, v in enumerate(values):\n        ax.text(x[j] + i * width, v + 0.02, f'{v:.2f}', ha='center', fontsize=9)\n\nplt.tight_layout()\nplt.savefig('detection_methods_comparison.png', dpi=150, bbox_inches='tight')\nplt.show()\n\n#%% [markdown]\n# ## ส่วนที่ 7: สรุปและ Best Practices\n\n#%%\nprint(\"=\" * 70)\nprint(\"📋 THRESHOLD SETTING BEST PRACTICES\")\nprint(\"=\" * 70)\n\nbest_practices = \"\"\"\n1️⃣ DOMAIN-SPECIFIC THRESHOLDS\n   - ไม่ใช้ default thresholds โดยไม่ validate\n   - ทดสอบกับ labeled data ถ้ามี\n   - ปรึกษากับ domain experts\n\n2️⃣ COST-BASED OPTIMIZATION\n   - พิจารณา cost ของ FP vs FN\n   - FN แพง → Lower threshold (more sensitive)\n   - FP แพง → Higher threshold (more conservative)\n\n3️⃣ ENSEMBLE APPROACH\n   - ใช้หลาย metrics ร่วมกัน\n   - Voting mechanism ลด false positives\n   - ถ้า metrics ไม่ agree → investigate further\n\n4️⃣ CONFIRMATION MECHANISM\n   - Require consecutive detections\n   - ป้องกัน temporary spikes\n   - Trade-off: delay detection\n\n5️⃣ PERIODIC REVIEW\n   - Review thresholds เป็นระยะ\n   - Data patterns อาจเปลี่ยน\n   - Business requirements อาจเปลี่ยน\n\n6️⃣ MONITORING & ALERTING\n   - Different severity levels\n   - Different thresholds for warning vs critical\n   - Escalation procedures\n\"\"\"\n\nprint(best_practices)\n\n#%% [markdown]\n# ## สรุป LAB 5\n# \n# ### สิ่งที่เรียนรู้:\n# 1. **Custom Metrics**: PSI, Wasserstein, Mean Shift, Combined Score\n# 2. **Threshold Optimization**: ใช้ labeled data หา optimal threshold\n# 3. **Cost-based Approach**: ปรับ threshold ตาม FP/FN costs\n# 4. **Robust Detection**: Ensemble + Confirmation mechanisms\n\n#%%\nprint(\"=\" * 60)\nprint(\"✅ LAB 5 COMPLETED!\")\nprint(\"=\" * 60)\nprint(\"\"\"\n📚 Key Takeaways:\n1. Default thresholds rarely optimal for your use case\n2. Use labeled data to optimize thresholds\n3. Consider business costs of FP vs FN\n4. Ensemble methods reduce false alarms\n5. Confirmation window adds robustness\n\n🔜 Next: LAB 6 - End-to-End Monitoring Pipeline\n\"\"\")\n```\n\n---\n\n## LAB 6: End-to-End Monitoring Pipeline\n\n```python\n#%% [markdown]\n# # LAB 6: End-to-End Monitoring Pipeline\n# ## สร้าง Pipeline สำหรับ Drift Monitoring แบบครบวงจร\n# \n# ### วัตถุประสงค์การเรียนรู้:\n# 1. รวมทุก components เข้าด้วยกัน\n# 2. สร้าง automated monitoring workflow\n# 3. Integrate กับ MLflow สำหรับ experiment tracking\n#\n# ### ทฤษฎี:\n# Production ML monitoring ต้องการ:\n# - Automated data ingestion\n# - Real-time drift detection\n# - Alerting mechanisms\n# - Experiment tracking\n# - Dashboard และ reporting\n\n#%% [markdown]\n# ## ส่วนที่ 1: เตรียม Environment\n\n#%%\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom datetime import datetime, timedelta\nfrom scipy import stats\nimport json\nimport os\nimport logging\nfrom collections import deque\nimport warnings\nwarnings.filterwarnings('ignore')\n\n# Setup logging\nlogging.basicConfig(\n    level=logging.INFO,\n    format='%(asctime)s - %(levelname)s - %(message)s'\n)\nlogger = logging.getLogger(__name__)\n\nnp.random.seed(42)\nplt.rcParams['figure.figsize'] = (14, 6)\n\nprint(\"✅ Libraries imported successfully!\")\n\n# สร้าง directory สำหรับ output\nos.makedirs('monitoring_output', exist_ok=True)\nos.makedirs('monitoring_output/reports', exist_ok=True)\nos.makedirs('monitoring_output/alerts', exist_ok=True)\n\n#%% [markdown]\n# ## ส่วนที่ 2: สร้าง Data Classes และ Utilities\n\n#%%\nfrom dataclasses import dataclass, field\nfrom typing import List, Dict, Optional, Callable\nfrom enum import Enum\n\nclass AlertSeverity(Enum):\n    INFO = \"info\"\n    WARNING = \"warning\"\n    CRITICAL = \"critical\"\n\nclass DriftType(Enum):\n    NONE = \"none\"\n    MILD = \"mild\"\n    MODERATE = \"moderate\"\n    SEVERE = \"severe\"\n\n@dataclass\nclass DriftResult:\n    \"\"\"ผลลัพธ์การตรวจจับ drift\"\"\"\n    timestamp: datetime\n    feature: str\n    drift_detected: bool\n    drift_type: DriftType\n    psi: float\n    ks_statistic: float\n    ks_pvalue: float\n    reference_mean: float\n    current_mean: float\n    reference_std: float\n    current_std: float\n    \n    def to_dict(self):\n        return {\n            'timestamp': self.timestamp.isoformat(),\n            'feature': self.feature,\n            'drift_detected': self.drift_detected,\n            'drift_type': self.drift_type.value,\n            'psi': self.psi,\n            'ks_statistic': self.ks_statistic,\n            'ks_pvalue': self.ks_pvalue,\n            'reference_mean': self.reference_mean,\n            'current_mean': self.current_mean,\n            'reference_std': self.reference_std,\n            'current_std': self.current_std\n        }\n\n@dataclass\nclass Alert:\n    \"\"\"Alert object\"\"\"\n    timestamp: datetime\n    severity: AlertSeverity\n    message: str\n    details: Dict\n    acknowledged: bool = False\n    \n    def to_dict(self):\n        return {\n            'timestamp': self.timestamp.isoformat(),\n            'severity': self.severity.value,\n            'message': self.message,\n            'details': self.details,\n            'acknowledged': self.acknowledged\n        }\n\n@dataclass\nclass MonitoringConfig:\n    \"\"\"Configuration สำหรับ monitoring\"\"\"\n    reference_window_size: int = 1000\n    current_window_size: int = 200\n    psi_mild_threshold: float = 0.1\n    psi_moderate_threshold: float = 0.2\n    psi_severe_threshold: float = 0.25\n    ks_significance: float = 0.05\n    check_interval_seconds: int = 60\n    alert_cooldown_minutes: int = 30\n    features_to_monitor: List[str] = field(default_factory=list)\n\n#%% [markdown]\n# ## ส่วนที่ 3: สร้าง Core Monitoring Components\n\n#%%\nclass DriftCalculator:\n    \"\"\"\n    Component สำหรับคำนวณ drift metrics\n    \"\"\"\n    \n    @staticmethod\n    def calculate_psi(reference: np.ndarray, current: np.ndarray, bins: int = 10) -> float:\n        \"\"\"Calculate Population Stability Index\"\"\"\n        breakpoints = np.percentile(reference, np.linspace(0, 100, bins + 1))\n        breakpoints = np.unique(breakpoints)\n        \n        if len(breakpoints) < 2:\n            return 0.0\n        \n        ref_counts, _ = np.histogram(reference, bins=breakpoints)\n        cur_counts, _ = np.histogram(current, bins=breakpoints)\n        \n        eps = 1e-6\n        ref_props = ref_counts / len(reference) + eps\n        cur_props = cur_counts / len(current) + eps\n        \n        psi = np.sum((cur_props - ref_props) * np.log(cur_props / ref_props))\n        return float(psi)\n    \n    @staticmethod\n    def calculate_ks_test(reference: np.ndarray, current: np.ndarray) -> tuple:\n        \"\"\"Calculate Kolmogorov-Smirnov test\"\"\"\n        statistic, pvalue = stats.ks_2samp(reference, current)\n        return float(statistic), float(pvalue)\n    \n    @staticmethod\n    def determine_drift_type(psi: float, config: MonitoringConfig) -> DriftType:\n        \"\"\"Determine drift severity based on PSI\"\"\"\n        if psi >= config.psi_severe_threshold:\n            return DriftType.SEVERE\n        elif psi >= config.psi_moderate_threshold:\n            return DriftType.MODERATE\n        elif psi >= config.psi_mild_threshold:\n            return DriftType.MILD\n        return DriftType.NONE\n\n#%%\nclass DataBuffer:\n    \"\"\"\n    Buffer สำหรับเก็บ reference และ current data\n    \"\"\"\n    \n    def __init__(self, config: MonitoringConfig):\n        self.config = config\n        self.reference_data: Dict[str, deque] = {}\n        self.current_data: Dict[str, deque] = {}\n        self.is_initialized = False\n        \n    def initialize(self, reference_df: pd.DataFrame):\n        \"\"\"Initialize with reference data\"\"\"\n        for feature in self.config.features_to_monitor:\n            if feature in reference_df.columns:\n                self.reference_data[feature] = deque(\n                    reference_df[feature].values[-self.config.reference_window_size:],\n                    maxlen=self.config.reference_window_size\n                )\n                self.current_data[feature] = deque(\n                    maxlen=self.config.current_window_size\n                )\n        self.is_initialized = True\n        logger.info(f\"DataBuffer initialized with {len(self.reference_data)} features\")\n    \n    def add_data(self, data: Dict[str, float]):\n        \"\"\"Add new data point\"\"\"\n        for feature, value in data.items():\n            if feature in self.current_data:\n                self.current_data[feature].append(value)\n    \n    def get_reference(self, feature: str) -> Optional[np.ndarray]:\n        \"\"\"Get reference data for a feature\"\"\"\n        if feature in self.reference_data:\n            return np.array(self.reference_data[feature])\n        return None\n    \n    def get_current(self, feature: str) -> Optional[np.ndarray]:\n        \"\"\"Get current data for a feature\"\"\"\n        if feature in self.current_data:\n            return np.array(self.current_data[feature])\n        return None\n    \n    def is_current_ready(self) -> bool:\n        \"\"\"Check if current buffer has enough data\"\"\"\n        for feature in self.config.features_to_monitor:\n            if feature in self.current_data:\n                if len(self.current_data[feature]) < self.config.current_window_size:\n                    return False\n        return True\n    \n    def update_reference(self):\n        \"\"\"Update reference with current data\"\"\"\n        for feature in self.config.features_to_monitor:\n            if feature in self.current_data and len(self.current_data[feature]) > 0:\n                # Add current data to reference\n                for val in self.current_data[feature]:\n                    self.reference_data[feature].append(val)\n                self.current_data[feature].clear()\n        logger.info(\"Reference data updated with current window\")\n\n#%%\nclass AlertManager:\n    \"\"\"\n    Component สำหรับจัดการ alerts\n    \"\"\"\n    \n    def __init__(self, config: MonitoringConfig):\n        self.config = config\n        self.alerts: List[Alert] = []\n        self.last_alert_time: Dict[str, datetime] = {}\n    \n    def should_alert(self, feature: str) -> bool:\n        \"\"\"Check if we should send alert (cooldown)\"\"\"\n        if feature not in self.last_alert_time:\n            return True\n        \n        elapsed = datetime.now() - self.last_alert_time[feature]\n        return elapsed > timedelta(minutes=self.config.alert_cooldown_minutes)\n    \n    def create_alert(self, drift_result: DriftResult) -> Optional[Alert]:\n        \"\"\"Create alert based on drift result\"\"\"\n        if not drift_result.drift_detected:\n            return None\n        \n        if not self.should_alert(drift_result.feature):\n            return None\n        \n        # Determine severity\n        if drift_result.drift_type == DriftType.SEVERE:\n            severity = AlertSeverity.CRITICAL\n        elif drift_result.drift_type == DriftType.MODERATE:\n            severity = AlertSeverity.WARNING\n        else:\n            severity = AlertSeverity.INFO\n        \n        message = (\n            f\"Drift detected in feature '{drift_result.feature}': \"\n            f\"PSI={drift_result.psi:.4f}, Type={drift_result.drift_type.value}\"\n        )\n        \n        alert = Alert(\n            timestamp=drift_result.timestamp,\n            severity=severity,\n            message=message,\n            details=drift_result.to_dict()\n        )\n        \n        self.alerts.append(alert)\n        self.last_alert_time[drift_result.feature] = datetime.now()\n        \n        # Log based on severity\n        if severity == AlertSeverity.CRITICAL:\n            logger.critical(message)\n        elif severity == AlertSeverity.WARNING:\n            logger.warning(message)\n        else:\n            logger.info(message)\n        \n        return alert\n    \n    def get_active_alerts(self) -> List[Alert]:\n        \"\"\"Get all unacknowledged alerts\"\"\"\n        return [a for a in self.alerts if not a.acknowledged]\n    \n    def acknowledge_alert(self, index: int):\n        \"\"\"Acknowledge an alert\"\"\"\n        if 0 <= index < len(self.alerts):\n            self.alerts[index].acknowledged = True\n    \n    def save_alerts(self, filepath: str):\n        \"\"\"Save alerts to file\"\"\"\n        alerts_data = [a.to_dict() for a in self.alerts]\n        with open(filepath, 'w') as f:\n            json.dump(alerts_data, f, indent=2)\n\n#%% [markdown]\n# ## ส่วนที่ 4: สร้าง Main Monitoring Pipeline\n\n#%%\nclass DriftMonitoringPipeline:\n    \"\"\"\n    Main pipeline สำหรับ drift monitoring\n    \"\"\"\n    \n    def __init__(self, config: MonitoringConfig):\n        self.config = config\n        self.data_buffer = DataBuffer(config)\n        self.alert_manager = AlertManager(config)\n        self.drift_calculator = DriftCalculator()\n        self.results_history: List[DriftResult] = []\n        self.is_running = False\n        \n    def initialize(self, reference_data: pd.DataFrame):\n        \"\"\"Initialize pipeline with reference data\"\"\"\n        self.data_buffer.initialize(reference_data)\n        logger.info(\"DriftMonitoringPipeline initialized\")\n    \n    def process_batch(self, batch_data: pd.DataFrame) -> List[DriftResult]:\n        \"\"\"Process a batch of new data\"\"\"\n        results = []\n        \n        # Add data to buffer\n        for idx, row in batch_data.iterrows():\n            data_point = {f: row[f] for f in self.config.features_to_monitor if f in row}\n            self.data_buffer.add_data(data_point)\n        \n        # Check if ready for drift detection\n        if not self.data_buffer.is_current_ready():\n            logger.debug(\"Current buffer not ready yet\")\n            return results\n        \n        # Perform drift detection for each feature\n        for feature in self.config.features_to_monitor:\n            result = self._detect_drift_for_feature(feature)\n            if result:\n                results.append(result)\n                self.results_history.append(result)\n                \n                # Create alert if needed\n                if result.drift_detected:\n                    self.alert_manager.create_alert(result)\n        \n        return results\n    \n    def _detect_drift_for_feature(self, feature: str) -> Optional[DriftResult]:\n        \"\"\"Detect drift for a single feature\"\"\"\n        reference = self.data_buffer.get_reference(feature)\n        current = self.data_buffer.get_current(feature)\n        \n        if reference is None or current is None:\n            return None\n        \n        if len(current) < 10:  # Need minimum samples\n            return None\n        \n        # Calculate metrics\n        psi = self.drift_calculator.calculate_psi(reference, current)\n        ks_stat, ks_pval = self.drift_calculator.calculate_ks_test(reference, current)\n        \n        # Determine drift type\n        drift_type = self.drift_calculator.determine_drift_type(psi, self.config)\n        drift_detected = drift_type != DriftType.NONE or ks_pval < self.config.ks_significance\n        \n        return DriftResult(\n            timestamp=datetime.now(),\n            feature=feature,\n            drift_detected=drift_detected,\n            drift_type=drift_type,\n            psi=psi,\n            ks_statistic=ks_stat,\n            ks_pvalue=ks_pval,\n            reference_mean=float(np.mean(reference)),\n            current_mean=float(np.mean(current)),\n            reference_std=float(np.std(reference)),\n            current_std=float(np.std(current))\n        )\n    \n    def get_summary_report(self) -> Dict:\n        \"\"\"Generate summary report\"\"\"\n        if not self.results_history:\n            return {'status': 'no_data'}\n        \n        # Group by feature\n        feature_summary = {}\n        for feature in self.config.features_to_monitor:\n            feature_results = [r for r in self.results_history if r.feature == feature]\n            if feature_results:\n                latest = feature_results[-1]\n                drift_count = sum(1 for r in feature_results if r.drift_detected)\n                feature_summary[feature] = {\n                    'latest_psi': latest.psi,\n                    'latest_drift_type': latest.drift_type.value,\n                    'drift_count': drift_count,\n                    'total_checks': len(feature_results),\n                    'drift_rate': drift_count / len(feature_results)\n                }\n        \n        return {\n            'timestamp': datetime.now().isoformat(),\n            'total_checks': len(self.results_history),\n            'total_drifts_detected': sum(1 for r in self.results_history if r.drift_detected),\n            'active_alerts': len(self.alert_manager.get_active_alerts()),\n            'feature_summary': feature_summary\n        }\n    \n    def save_results(self, output_dir: str = 'monitoring_output'):\n        \"\"\"Save all results to files\"\"\"\n        # Save drift results\n        results_data = [r.to_dict() for r in self.results_history]\n        with open(f'{output_dir}/drift_results.json', 'w') as f:\n            json.dump(results_data, f, indent=2)\n        \n        # Save alerts\n        self.alert_manager.save_alerts(f'{output_dir}/alerts/alerts.json')\n        \n        # Save summary\n        summary = self.get_summary_report()\n        with open(f'{output_dir}/reports/summary.json', 'w') as f:\n            json.dump(summary, f, indent=2)\n        \n        logger.info(f\"Results saved to {output_dir}\")\n\n#%% [markdown]\n# ## ส่วนที่ 5: สร้าง Report Generator\n\n#%%\nclass ReportGenerator:\n    \"\"\"\n    Component สำหรับสร้าง reports และ visualizations\n    \"\"\"\n    \n    def __init__(self, pipeline: DriftMonitoringPipeline):\n        self.pipeline = pipeline\n    \n    def generate_dashboard_data(self) -> Dict:\n        \"\"\"Generate data for dashboard\"\"\"\n        summary = self.pipeline.get_summary_report()\n        \n        # Time series data for each feature\n        time_series = {}\n        for feature in self.pipeline.config.features_to_monitor:\n            feature_results = [\n                r for r in self.pipeline.results_history \n                if r.feature == feature\n            ]\n            time_series[feature] = {\n                'timestamps': [r.timestamp.isoformat() for r in feature_results],\n                'psi_values': [r.psi for r in feature_results],\n                'drift_types': [r.drift_type.value for r in feature_results]\n            }\n        \n        return {\n            'summary': summary,\n            'time_series': time_series,\n            'alerts': [a.to_dict() for a in self.pipeline.alert_manager.get_active_alerts()]\n        }\n    \n    def plot_drift_trends(self, save_path: str = None):\n        \"\"\"Plot drift trends for all features\"\"\"\n        n_features = len(self.pipeline.config.features_to_monitor)\n        \n        if n_features == 0:\n            logger.warning(\"No features to plot\")\n            return\n        \n        fig, axes = plt.subplots(n_features, 1, figsize=(14, 4*n_features), sharex=True)\n        \n        if n_features == 1:\n            axes = [axes]\n        \n        colors = {'none': 'green', 'mild': 'yellow', 'moderate': 'orange', 'severe': 'red'}\n        \n        for ax, feature in zip(axes, self.pipeline.config.features_to_monitor):\n            feature_results = [\n                r for r in self.pipeline.results_history \n                if r.feature == feature\n            ]\n            \n            if not feature_results:\n                continue\n            \n            timestamps = [r.timestamp for r in feature_results]\n            psi_values = [r.psi for r in feature_results]\n            drift_types = [r.drift_type.value for r in feature_results]\n            \n            # Plot PSI\n            scatter_colors = [colors.get(dt, 'gray') for dt in drift_types]\n            ax.scatter(timestamps, psi_values, c=scatter_colors, s=30, alpha=0.7)\n            ax.plot(timestamps, psi_values, 'b-', alpha=0.3)\n            \n            # Add thresholds\n            ax.axhline(self.pipeline.config.psi_mild_threshold, \n                      color='yellow', linestyle='--', alpha=0.7, label='Mild')\n            ax.axhline(self.pipeline.config.psi_moderate_threshold, \n                      color='orange', linestyle='--', alpha=0.7, label='Moderate')\n            ax.axhline(self.pipeline.config.psi_severe_threshold, \n                      color='red', linestyle='--', alpha=0.7, label='Severe')\n            \n            ax.set_ylabel(f'{feature}\\nPSI')\n            ax.legend(loc='upper right')\n            ax.grid(True, alpha=0.3)\n        \n        axes[-1].set_xlabel('Time')\n        plt.suptitle('Drift Monitoring Dashboard', fontsize=14, fontweight='bold')\n        plt.tight_layout()\n        \n        if save_path:\n            plt.savefig(save_path, dpi=150, bbox_inches='tight')\n            logger.info(f\"Plot saved to {save_path}\")\n        \n        plt.show()\n    \n    def generate_html_report(self, output_path: str):\n        \"\"\"Generate HTML report\"\"\"\n        summary = self.pipeline.get_summary_report()\n        \n        html_content = f\"\"\"\n        <!DOCTYPE html>\n        <html>\n        <head>\n            <title>Drift Monitoring Report</title>\n            <style>\n                body {{ font-family: Arial, sans-serif; margin: 20px; }}\n                .header {{ background-color: #2196F3; color: white; padding: 20px; }}\n                .summary {{ background-color: #f0f0f0; padding: 15px; margin: 10px 0; }}\n                .alert-critical {{ background-color: #ffebee; border-left: 4px solid #f44336; padding: 10px; margin: 5px 0; }}\n                .alert-warning {{ background-color: #fff3e0; border-left: 4px solid #ff9800; padding: 10px; margin: 5px 0; }}\n                .alert-info {{ background-color: #e3f2fd; border-left: 4px solid #2196F3; padding: 10px; margin: 5px 0; }}\n                table {{ border-collapse: collapse; width: 100%; }}\n                th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}\n                th {{ background-color: #2196F3; color: white; }}\n            </style>\n        </head>\n        <body>\n            <div class=\"header\">\n                <h1>🔍 Drift Monitoring Report</h1>\n                <p>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>\n            </div>\n            \n            <div class=\"summary\">\n                <h2>📊 Summary</h2>\n                <p><strong>Total Checks:</strong> {summary.get('total_checks', 0)}</p>\n                <p><strong>Total Drifts Detected:</strong> {summary.get('total_drifts_detected', 0)}</p>\n                <p><strong>Active Alerts:</strong> {summary.get('active_alerts', 0)}</p>\n            </div>\n            \n            <h2>📈 Feature Summary</h2>\n            <table>\n                <tr>\n                    <th>Feature</th>\n                    <th>Latest PSI</th>\n                    <th>Drift Type</th>\n                    <th>Drift Count</th>\n                    <th>Drift Rate</th>\n                </tr>\n        \"\"\"\n        \n        for feature, data in summary.get('feature_summary', {}).items():\n            html_content += f\"\"\"\n                <tr>\n                    <td>{feature}</td>\n                    <td>{data['latest_psi']:.4f}</td>\n                    <td>{data['latest_drift_type']}</td>\n                    <td>{data['drift_count']}</td>\n                    <td>{data['drift_rate']:.1%}</td>\n                </tr>\n            \"\"\"\n        \n        html_content += \"\"\"\n            </table>\n            \n            <h2>⚠️ Active Alerts</h2>\n        \"\"\"\n        \n        for alert in self.pipeline.alert_manager.get_active_alerts():\n            alert_class = f\"alert-{alert.severity.value}\"\n            html_content += f\"\"\"\n            <div class=\"{alert_class}\">\n                <strong>{alert.severity.value.upper()}</strong>: {alert.message}\n                <br><small>{alert.timestamp.strftime('%Y-%m-%d %H:%M:%S')}</small>\n            </div>\n            \"\"\"\n        \n        html_content += \"\"\"\n        </body>\n        </html>\n        \"\"\"\n        \n        with open(output_path, 'w') as f:\n            f.write(html_content)\n        \n        logger.info(f\"HTML report saved to {output_path}\")\n\n#%% [markdown]\n# ## ส่วนที่ 6: ทดสอบ Full Pipeline\n\n#%%\n# สร้าง test data\ndef generate_test_data(n_samples=5000):\n    \"\"\"สร้างข้อมูลทดสอบที่มี drift\"\"\"\n    np.random.seed(42)\n    \n    data = []\n    base_means = {'feature_a': 50, 'feature_b': 100, 'feature_c': 75}\n    \n    for i in range(n_samples):\n        # สร้าง drift ที่ feature_a หลัง sample 3000\n        if i < 3000:\n            feature_a_mean = base_means['feature_a']\n        else:\n            # Gradual drift\n            progress = (i - 3000) / 2000\n            feature_a_mean = base_means['feature_a'] + progress * 20\n        \n        # สร้าง sudden drift ที่ feature_b ที่ sample 2000\n        if i < 2000:\n            feature_b_mean = base_means['feature_b']\n        else:\n            feature_b_mean = base_means['feature_b'] + 30\n        \n        # feature_c ไม่มี drift\n        feature_c_mean = base_means['feature_c']\n        \n        data.append({\n            'timestamp': datetime(2024, 1, 1) + timedelta(hours=i),\n            'feature_a': np.random.normal(feature_a_mean, 10),\n            'feature_b': np.random.normal(feature_b_mean, 15),\n            'feature_c': np.random.normal(feature_c_mean, 12)\n        })\n    \n    return pd.DataFrame(data)\n\n# สร้างข้อมูลทดสอบ\ntest_data = generate_test_data(5000)\nprint(f\"📊 Test data shape: {test_data.shape}\")\nprint(test_data.head())\n\n#%%\n# Visualize test data\nfig, axes = plt.subplots(3, 1, figsize=(14, 8), sharex=True)\n\nfor ax, feature in zip(axes, ['feature_a', 'feature_b', 'feature_c']):\n    ax.plot(test_data['timestamp'], test_data[feature], alpha=0.3)\n    rolling_mean = test_data[feature].rolling(100).mean()\n    ax.plot(test_data['timestamp'], rolling_mean, 'r-', linewidth=2, label='Rolling Mean')\n    ax.set_ylabel(feature)\n    ax.legend()\n\naxes[0].set_title('Test Data with Drift Patterns')\naxes[-1].set_xlabel('Time')\nplt.tight_layout()\nplt.show()\n\n#%%\n# Configure และ run pipeline\nconfig = MonitoringConfig(\n    reference_window_size=1000,\n    current_window_size=200,\n    psi_mild_threshold=0.1,\n    psi_moderate_threshold=0.2,\n    psi_severe_threshold=0.25,\n    ks_significance=0.05,\n    features_to_monitor=['feature_a', 'feature_b', 'feature_c']\n)\n\n# สร้าง pipeline\npipeline = DriftMonitoringPipeline(config)\n\n# ใช้ 1000 samples แรกเป็น reference\nreference_data = test_data.iloc[:1000]\npipeline.initialize(reference_data)\n\nprint(\"=\" * 60)\nprint(\"🚀 Starting Drift Monitoring Pipeline\")\nprint(\"=\" * 60)\n\n# Process data เป็น batches\nbatch_size = 200\nremaining_data = test_data.iloc[1000:]\n\nfor i in range(0, len(remaining_data), batch_size):\n    batch = remaining_data.iloc[i:i+batch_size]\n    results = pipeline.process_batch(batch)\n    \n    if results and any(r.drift_detected for r in results):\n        print(f\"\\n📍 Batch {i//batch_size + 1} results:\")\n        for r in results:\n            if r.drift_detected:\n                print(f\"   ⚠️ {r.feature}: PSI={r.psi:.4f}, Type={r.drift_type.value}\")\n\n#%%\n# Generate summary\nprint(\"\\n\" + \"=\" * 60)\nprint(\"📊 FINAL SUMMARY REPORT\")\nprint(\"=\" * 60)\n\nsummary = pipeline.get_summary_report()\nprint(f\"\\nTotal Checks: {summary['total_checks']}\")\nprint(f\"Total Drifts Detected: {summary['total_drifts_detected']}\")\nprint(f\"Active Alerts: {summary['active_alerts']}\")\n\nprint(\"\\nFeature Summary:\")\nfor feature, data in summary['feature_summary'].items():\n    print(f\"\\n  {feature}:\")\n    print(f\"    Latest PSI: {data['latest_psi']:.4f}\")\n    print(f\"    Drift Type: {data['latest_drift_type']}\")\n    print(f\"    Drift Count: {data['drift_count']}/{data['total_checks']}\")\n    print(f\"    Drift Rate: {data['drift_rate']:.1%}\")\n\n#%%\n# Generate reports\nreport_generator = ReportGenerator(pipeline)\n\n# Plot trends\nreport_generator.plot_drift_trends(save_path='monitoring_output/drift_trends.png')\n\n# Generate HTML report\nreport_generator.generate_html_report('monitoring_output/reports/drift_report.html')\n\n# Save all results\npipeline.save_results()\n\nprint(\"\\n✅ All reports generated and saved to monitoring_output/\")\n\n#%% [markdown]\n# ## ส่วนที่ 7: Integration กับ MLflow (Optional)\n\n#%%\n# Note: ส่วนนี้ต้อง install mlflow ก่อน: pip install mlflow\n\ntry:\n    import mlflow\n    MLFLOW_AVAILABLE = True\nexcept ImportError:\n    MLFLOW_AVAILABLE = False\n    print(\"⚠️ MLflow not installed. Skipping MLflow integration.\")\n    print(\"   Install with: pip install mlflow\")\n\nif MLFLOW_AVAILABLE:\n    class MLflowDriftTracker:\n        \"\"\"\n        Integration กับ MLflow สำหรับ track drift experiments\n        \"\"\"\n        \n        def __init__(self, experiment_name: str = \"drift_monitoring\"):\n            mlflow.set_experiment(experiment_name)\n            self.active_run = None\n        \n        def start_run(self, run_name: str = None):\n            \"\"\"Start new MLflow run\"\"\"\n            self.active_run = mlflow.start_run(run_name=run_name)\n            return self.active_run\n        \n        def log_drift_result(self, result: DriftResult):\n            \"\"\"Log drift result to MLflow\"\"\"\n            if self.active_run is None:\n                return\n            \n            # Log metrics\n            mlflow.log_metric(f\"{result.feature}_psi\", result.psi)\n            mlflow.log_metric(f\"{result.feature}_ks_stat\", result.ks_statistic)\n            mlflow.log_metric(f\"{result.feature}_drift\", 1 if result.drift_detected else 0)\n            \n            # Log params\n            mlflow.log_param(f\"{result.feature}_drift_type\", result.drift_type.value)\n        \n        def log_summary(self, summary: Dict):\n            \"\"\"Log summary to MLflow\"\"\"\n            if self.active_run is None:\n                return\n            \n            mlflow.log_metric(\"total_drifts\", summary.get('total_drifts_detected', 0))\n            mlflow.log_metric(\"total_checks\", summary.get('total_checks', 0))\n            \n            for feature, data in summary.get('feature_summary', {}).items():\n                mlflow.log_metric(f\"{feature}_drift_rate\", data['drift_rate'])\n        \n        def log_artifact(self, artifact_path: str):\n            \"\"\"Log artifact to MLflow\"\"\"\n            if self.active_run is None:\n                return\n            mlflow.log_artifact(artifact_path)\n        \n        def end_run(self):\n            \"\"\"End MLflow run\"\"\"\n            if self.active_run:\n                mlflow.end_run()\n                self.active_run = None\n    \n    # Example usage\n    print(\"\\n\" + \"=\" * 60)\n    print(\"📊 Logging to MLflow\")\n    print(\"=\" * 60)\n    \n    tracker = MLflowDriftTracker(\"drift_monitoring_lab\")\n    tracker.start_run(run_name=\"pipeline_run_1\")\n    \n    # Log summary\n    tracker.log_summary(summary)\n    \n    # Log artifacts\n    tracker.log_artifact('monitoring_output/drift_trends.png')\n    tracker.log_artifact('monitoring_output/reports/drift_report.html')\n    \n    tracker.end_run()\n    print(\"✅ Results logged to MLflow\")\n\n#%% [markdown]\n# ## สรุป LAB 6\n# \n# ### สิ่งที่เรียนรู้:\n# 1. **Pipeline Architecture**: แยก components ชัดเจน\n# 2. **Data Classes**: ใช้ dataclasses สำหรับ type safety\n# 3. **Alert Management**: จัดการ alerts พร้อม cooldown\n# 4. **Report Generation**: HTML reports และ visualizations\n# 5. **MLflow Integration**: Track experiments\n\n#%%\nprint(\"=\" * 60)\nprint(\"✅ LAB 6 COMPLETED!\")\nprint(\"=\" * 60)\nprint(\"\"\"\n📚 Key Takeaways:\n1. Modular architecture ทำให้ maintain ง่าย\n2. Data buffering ช่วย handle streaming data\n3. Alert management ป้องกัน alert fatigue\n4. Automated reporting saves time\n5. MLflow integration enables experiment tracking\n\n🎉 CONGRATULATIONS! You have completed all 6 labs!\n\n📋 What you've learned:\n- LAB 1: Data Drift Concepts (Covariate/Concept Shift)\n- LAB 2: Feature Drift Detection (per-feature analysis)\n- LAB 3: Multivariate Drift Analysis (correlation/PCA)\n- LAB 4: Production Simulation (streaming detection)\n- LAB 5: Custom Metrics & Thresholds (optimization)\n- LAB 6: End-to-End Pipeline (production-ready)\n\n🚀 Next steps:\n- Deploy pipeline to production\n- Add more sophisticated alerting (email, Slack)\n- Integrate with model retraining triggers\n- Add A/B testing capabilities\n\"\"\")\n\n#%%\n# Final cleanup and summary\nprint(\"\\n📁 Output files created:\")\nfor root, dirs, files in os.walk('monitoring_output'):\n    for file in files:\n        filepath = os.path.join(root, file)\n        print(f\"   {filepath}\")\n```\n\n---\n\n## สรุปรวม\n\nทั้ง 6 Labs ครอบคลุมเนื้อหาดังนี้:\n\n| Lab | หัวข้อ | สิ่งที่เรียนรู้ |\n|-----|--------|----------------|\n| 1 | Understanding Data Drift | Covariate/Concept Shift, KS/PSI/Wasserstein |\n| 2 | Feature Drift Detection | Per-feature analysis, Numerical vs Categorical |\n| 3 | Multivariate Drift | Correlation, PCA, Mahalanobis Distance |\n| 4 | Production Simulation | Streaming, Sliding Window, Page-Hinkley |\n| 5 | Custom Metrics & Thresholds | Optimization, Cost-based, Ensemble |\n| 6 | End-to-End Pipeline | Architecture, Alerting, Reporting, MLflow |"
    }
  ],
}
 


In [4]:
# Simple extraction
text = response["content"][0]["text"]
print(text)

# Save to file
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(text)

# Complete MLOps Labs: Understanding Data Drift Concepts with sklearn

## LAB 1: Understanding Data Drift Concepts

```python
#%% [markdown]
# # LAB 1: Understanding Data Drift Concepts
# ## ทำความเข้าใจแนวคิดพื้นฐานของ Data Drift
# 
# ### วัตถุประสงค์การเรียนรู้:
# 1. เข้าใจความแตกต่างระหว่าง Covariate Shift และ Concept Drift
# 2. เรียนรู้ Statistical tests สำหรับ drift detection (KS, PSI, Wasserstein)
# 3. สามารถเลือก drift detection method ที่เหมาะสมกับสถานการณ์ต่างๆ
#
# ### ทฤษฎีพื้นฐาน:
# **Data Drift** คือ การเปลี่ยนแปลงของข้อมูลเมื่อเวลาผ่านไป ซึ่งอาจส่งผลกระทบต่อประสิทธิภาพของ ML Model
#
# มี 2 ประเภทหลัก:
# - **Covariate Shift**: การเปลี่ยนแปลงของ input features P(X) โดยที่ความสัมพันธ์ P(Y|X) ยังคงเดิม
# - **Concept Drift**: การเปลี่ยนแปลงของความสัมพันธ์ระหว่าง input และ output P(Y|X)

#%% [markdown]
# ## ส่วนที่ 1: เตรียม Environment และ Import Libraries
# 
# ก่อนเริ่มต้น เราจะ import libraries ที่จำเป็นทั้งหมด

#%%
# Import libraries ที่จำเป็น
import numpy as np
import panda